<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_11_Deliberative_Reasoning_Smart_Decision_Making.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In Tutorial 10, you built sophisticated memory systems for context and learning. Now we're adding deliberative reasoning - the ability to systematically evaluate options, weigh trade-offs, and make informed decisions through structured analysis.

What you'll build:

• Multi-criteria decision analysis framework

• Option generation and evaluation systems

• Reasoning strategy selection and adaptation

• Deliberative agents with structured decision processes

• Risk assessment and uncertainty handling

• Decision explanation and justification systems

Why this matters:

Intelligent systems need to make complex decisions under uncertainty, balancing multiple criteria and constraints. Deliberative reasoning enables agents to move beyond reactive responses to thoughtful, strategic decision-making that can be explained and audited.

By the end, you'll understand:

• How to structure complex decision problems

• Multi-criteria evaluation and trade-off analysis

• Reasoning strategy patterns and when to use them

• Uncertainty quantification and risk management

• Decision explanation and transparency mechanisms

In [1]:
print("Tutorial 11: Deliberative Reasoning - Smart Decision Making")
print("=" * 58)
print()
print("Building intelligent decision-making systems with structured reasoning...")
print()

Tutorial 11: Deliberative Reasoning - Smart Decision Making

Building intelligent decision-making systems with structured reasoning...



In [4]:
# Essential imports
import uuid
import time
import math
import random
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple, Callable
from enum import Enum
from collections import defaultdict

In [5]:
# Basic decision-making types and concepts
class DecisionType(Enum):
    """Categories of decisions based on complexity and stakes"""
    OPERATIONAL = "operational"        # Day-to-day operational choices
    TACTICAL = "tactical"             # Medium-term planning decisions
    STRATEGIC = "strategic"           # Long-term strategic choices
    EMERGENCY = "emergency"           # Time-critical urgent decisions

class ReasoningStrategy(Enum):
    """Different approaches to decision making"""
    ANALYTICAL = "analytical"         # Systematic analysis of all options
    INTUITIVE = "intuitive"          # Fast, experience-based decisions
    SATISFICING = "satisficing"      # Find first acceptable solution
    OPTIMIZING = "optimizing"        # Find best possible solution

@dataclass
class DecisionCriterion:
    """
    A criterion for evaluating decision options

    Think of this as "what matters" in your decision.
    For buying a car: cost, safety, fuel efficiency, etc.
    """
    id: str
    name: str
    description: str
    weight: float = 1.0                    # How important is this? (0-1)
    direction: str = "maximize"            # "maximize" (higher better) or "minimize" (lower better)

    def evaluate(self, option_data: Any) -> float:
        """
        Score an option on this criterion (0-1 scale)

        This is where domain knowledge goes - how do you
        measure "good" for this specific criterion?
        """
        if isinstance(option_data, (int, float)):
            # Simple numeric evaluation
            if self.direction == "minimize":
                # For cost: lower is better, so invert
                return max(0.0, min(1.0, 1.0 - (option_data / 100000)))
            else:
                # For quality: higher is better
                return max(0.0, min(1.0, option_data / 100))

        return 0.5  # Default for unknown types

@dataclass
class DecisionOption:
    """
    A possible choice in a decision

    This represents one alternative you could choose,
    with all the data needed to evaluate it.
    """
    id: str
    name: str
    description: str
    attributes: Dict[str, Any] = field(default_factory=dict)

    # Results from evaluation (filled in later)
    scores: Dict[str, float] = field(default_factory=dict)
    overall_score: float = 0.0

@dataclass
class DecisionProblem:
    """
    A complete decision that needs to be made

    This packages together:
    - What you're deciding (title/description)
    - What matters (criteria)
    - What you can choose (options)
    """
    id: str
    title: str
    description: str
    decision_type: DecisionType

    criteria: List[DecisionCriterion] = field(default_factory=list)
    options: List[DecisionOption] = field(default_factory=list)
    selected_option: Optional[str] = None

class BasicEvaluator:
    """
    Simple multi-criteria decision evaluator

    This demonstrates the core concept: score each option
    on each criterion, then combine using weights.
    """

    def evaluate_decision(self, problem: DecisionProblem) -> List[DecisionOption]:
        """
        Evaluate all options and return them ranked by score

        This is the heart of multi-criteria decision analysis:
        1. Score each option on each criterion
        2. Weight the scores by importance
        3. Sum to get overall score
        4. Rank options by overall score
        """
        print(f"   📊 Evaluating {len(problem.options)} options against {len(problem.criteria)} criteria...")

        # Step 1 & 2: Score and weight
        for option in problem.options:
            total_weighted_score = 0.0
            total_weight = 0.0

            for criterion in problem.criteria:
                # Get data for this criterion
                data = self._extract_criterion_data(option, criterion)

                # Score it
                score = criterion.evaluate(data)
                option.scores[criterion.id] = score

                # Add to weighted total
                total_weighted_score += score * criterion.weight
                total_weight += criterion.weight

            # Step 3: Calculate overall score
            option.overall_score = total_weighted_score / total_weight if total_weight > 0 else 0.0

        # Step 4: Sort by score (highest first)
        ranked_options = sorted(problem.options, key=lambda x: x.overall_score, reverse=True)

        print(f"   ✅ Evaluation complete")
        return ranked_options

    def _extract_criterion_data(self, option: DecisionOption, criterion: DecisionCriterion) -> Any:
        """Extract the right data from an option for a criterion"""
        # Try exact match first
        if criterion.id in option.attributes:
            return option.attributes[criterion.id]

        # Try name-based match
        criterion_key = criterion.name.lower().replace(' ', '_')
        if criterion_key in option.attributes:
            return option.attributes[criterion_key]

        # Default fallback
        return 50  # Neutral value

class ReasoningEngine:
    """
    Demonstrates different reasoning strategies

    Key insight: Different problems need different approaches!
    Emergency → fast intuitive decisions
    Strategic → thorough analytical decisions
    """

    def __init__(self):
        self.evaluator = BasicEvaluator()
        self.strategies = {
            ReasoningStrategy.ANALYTICAL: self._analytical_strategy,
            ReasoningStrategy.INTUITIVE: self._intuitive_strategy,
            ReasoningStrategy.SATISFICING: self._satisficing_strategy,
            ReasoningStrategy.OPTIMIZING: self._optimizing_strategy
        }

    def solve_problem(self, problem: DecisionProblem, strategy: ReasoningStrategy = None) -> Optional[DecisionOption]:
        """
        Solve a decision problem using specified reasoning strategy

        If no strategy given, automatically choose based on problem type
        (this is called "meta-reasoning" - reasoning about reasoning!)
        """
        if strategy is None:
            strategy = self._choose_strategy(problem)

        print(f"   🧠 Using {strategy.value} reasoning...")

        # Apply the chosen strategy
        strategy_func = self.strategies[strategy]
        return strategy_func(problem)

    def _choose_strategy(self, problem: DecisionProblem) -> ReasoningStrategy:
        """
        Choose reasoning strategy based on problem characteristics

        This demonstrates meta-cognition: thinking about how to think!
        """
        if problem.decision_type == DecisionType.EMERGENCY:
            return ReasoningStrategy.INTUITIVE  # Need speed!
        elif problem.decision_type == DecisionType.STRATEGIC:
            return ReasoningStrategy.ANALYTICAL  # Need thoroughness!
        elif len(problem.criteria) > 5:
            return ReasoningStrategy.OPTIMIZING  # Complex problem
        else:
            return ReasoningStrategy.SATISFICING  # Good enough

    def _analytical_strategy(self, problem: DecisionProblem) -> Optional[DecisionOption]:
        """
        Systematic, thorough analysis of all options

        Best for: Important decisions with time to think
        Process: Evaluate everything carefully, check for robustness
        """
        print("     📋 Systematic evaluation of all options...")

        # Evaluate all options thoroughly
        ranked_options = self.evaluator.evaluate_decision(problem)

        # Additional analysis: sensitivity check
        print("     🔍 Checking robustness of decision...")
        if len(ranked_options) >= 2:
            top_score = ranked_options[0].overall_score
            second_score = ranked_options[1].overall_score

            if top_score - second_score < 0.1:  # Very close scores
                print("     ⚠️  Top options are very close - decision is sensitive")
            else:
                print("     ✅ Clear winner identified")

        return ranked_options[0] if ranked_options else None

    def _intuitive_strategy(self, problem: DecisionProblem) -> Optional[DecisionOption]:
        """
        Fast, heuristic-based decisions

        Best for: Time pressure, emergency situations
        Process: Use simple rules and gut feelings
        """
        print("     ⚡ Quick heuristic evaluation...")

        if not problem.options:
            return None

        # Use simple heuristics instead of full evaluation
        best_option = None
        best_heuristic_score = -1

        for option in problem.options:
            # Simple heuristic: focus on most important criterion
            if problem.criteria:
                main_criterion = max(problem.criteria, key=lambda c: c.weight)
                data = self.evaluator._extract_criterion_data(option, main_criterion)
                heuristic_score = main_criterion.evaluate(data)

                # Add some "gut feeling" randomness
                heuristic_score += random.uniform(-0.1, 0.1)

                if heuristic_score > best_heuristic_score:
                    best_heuristic_score = heuristic_score
                    best_option = option

        return best_option

    def _satisficing_strategy(self, problem: DecisionProblem) -> Optional[DecisionOption]:
        """
        Find first "good enough" solution

        Best for: Routine decisions, time constraints
        Process: Stop at first acceptable option
        """
        print("     ✋ Looking for first acceptable option...")

        acceptable_threshold = 0.6  # "Good enough" threshold

        for option in problem.options:
            # Quick evaluation of just this option
            temp_problem = DecisionProblem(
                id="temp", title="temp", description="temp",
                decision_type=problem.decision_type,
                criteria=problem.criteria,
                options=[option]
            )

            ranked = self.evaluator.evaluate_decision(temp_problem)
            if ranked and ranked[0].overall_score >= acceptable_threshold:
                print(f"     ✅ Found acceptable option: {option.name}")
                return option

        # If nothing acceptable, return best available
        print("     📉 No option met threshold, taking best available...")
        ranked_all = self.evaluator.evaluate_decision(problem)
        return ranked_all[0] if ranked_all else None

    def _optimizing_strategy(self, problem: DecisionProblem) -> Optional[DecisionOption]:
        """
        Find the absolute best solution

        Best for: Critical decisions with resources for thoroughness
        Process: Try multiple approaches, find consensus
        """
        print("     🎯 Optimizing: trying multiple evaluation approaches...")

        # Method 1: Standard evaluation
        ranked_v1 = self.evaluator.evaluate_decision(problem)

        # Method 2: Boost most important criterion
        if problem.criteria:
            # Temporarily increase weight of top criterion
            main_criterion = max(problem.criteria, key=lambda c: c.weight)
            original_weight = main_criterion.weight
            main_criterion.weight *= 1.5

            # Renormalize weights
            total_weight = sum(c.weight for c in problem.criteria)
            for c in problem.criteria:
                c.weight /= total_weight

            ranked_v2 = self.evaluator.evaluate_decision(problem)

            # Restore original weight
            main_criterion.weight = original_weight
            # Restore other weights
            for c in problem.criteria:
                if c != main_criterion:
                    c.weight *= total_weight
        else:
            ranked_v2 = ranked_v1

        # Find consensus choice
        print("     🤝 Finding consensus across methods...")
        if ranked_v1 and ranked_v2:
            if ranked_v1[0].id == ranked_v2[0].id:
                print("     ✅ Both methods agree!")
                return ranked_v1[0]
            else:
                print("     🔄 Methods disagree, using weighted approach...")
                # Could implement more sophisticated consensus here
                return ranked_v1[0]  # Default to first method

        return ranked_v1[0] if ranked_v1 else None

class LearningAgent:
    """
    Agent that learns from decision-making experience

    This demonstrates how agents can improve over time by:
    1. Tracking which strategies work best
    2. Recognizing problem patterns
    3. Adapting strategy selection
    """

    def __init__(self, name: str):
        self.name = name
        self.reasoning_engine = ReasoningEngine()

        # Learning components
        self.decision_history = []
        self.strategy_performance = defaultdict(lambda: {'attempts': 0, 'successes': 0})
        self.problem_patterns = defaultdict(lambda: {'count': 0, 'best_strategy': None})

        print(f"🤖 Created learning agent: {self.name}")

    def make_decision(self, problem: DecisionProblem) -> Optional[DecisionOption]:
        """Make a decision and learn from the experience"""
        print(f"🤖 {self.name} deciding on: {problem.title}")

        # Choose strategy (this gets smarter over time!)
        strategy = self._choose_strategy_intelligently(problem)

        # Make decision
        start_time = time.time()
        result = self.reasoning_engine.solve_problem(problem, strategy)
        decision_time = time.time() - start_time

        # Learn from this experience
        self._learn_from_decision(problem, result, strategy, decision_time)

        return result

    def _choose_strategy_intelligently(self, problem: DecisionProblem) -> ReasoningStrategy:
        """Choose strategy based on learned experience"""

        # Identify problem pattern
        pattern = f"{problem.decision_type.value}_{len(problem.criteria)}_{len(problem.options)}"

        # If we've seen this pattern before, use best known strategy
        if pattern in self.problem_patterns and self.problem_patterns[pattern]['count'] > 2:
            best_strategy = self.problem_patterns[pattern]['best_strategy']
            if best_strategy:
                print(f"   🧠 Using learned strategy for pattern '{pattern}': {best_strategy.value}")
                return best_strategy

        # Otherwise, use strategy with best overall performance
        if self.strategy_performance:
            best_strategy = max(self.strategy_performance.keys(),
                              key=lambda s: self._calculate_success_rate(s))
            print(f"   📈 Using best-performing strategy: {best_strategy.value}")
            return best_strategy

        # Fallback to default strategy selection
        return self.reasoning_engine._choose_strategy(problem)

    def _calculate_success_rate(self, strategy: ReasoningStrategy) -> float:
        """Calculate success rate for a strategy"""
        perf = self.strategy_performance[strategy]
        if perf['attempts'] == 0:
            return 0.0
        return perf['successes'] / perf['attempts']

    def _learn_from_decision(self, problem: DecisionProblem, result: Optional[DecisionOption],
                           strategy: ReasoningStrategy, decision_time: float):
        """Update learning based on decision outcome"""

        # Record the experience
        experience = {
            'problem_type': problem.decision_type.value,
            'strategy': strategy.value,
            'success': result is not None,
            'decision_time': decision_time,
            'result_score': result.overall_score if result else 0.0
        }
        self.decision_history.append(experience)

        # Update strategy performance
        self.strategy_performance[strategy]['attempts'] += 1
        if result and result.overall_score > 0.6:  # Define "success"
            self.strategy_performance[strategy]['successes'] += 1

        # Update problem pattern knowledge
        pattern = f"{problem.decision_type.value}_{len(problem.criteria)}_{len(problem.options)}"
        pattern_data = self.problem_patterns[pattern]
        pattern_data['count'] += 1

        # If this strategy performed well, consider it the best for this pattern
        if result and result.overall_score > 0.7:
            pattern_data['best_strategy'] = strategy

    def get_learning_summary(self) -> Dict[str, Any]:
        """Get summary of what the agent has learned"""
        total_decisions = len(self.decision_history)
        if total_decisions == 0:
            return {'message': 'No decisions made yet'}

        # Calculate overall success rate
        successes = sum(1 for exp in self.decision_history if exp['success'])
        overall_success_rate = successes / total_decisions

        # Find best strategy
        best_strategy = None
        best_rate = 0.0
        for strategy, perf in self.strategy_performance.items():
            rate = self._calculate_success_rate(strategy)
            if rate > best_rate:
                best_rate = rate
                best_strategy = strategy

        return {
            'total_decisions': total_decisions,
            'overall_success_rate': overall_success_rate,
            'best_strategy': best_strategy.value if best_strategy else None,
            'best_strategy_success_rate': best_rate,
            'patterns_learned': len(self.problem_patterns),
            'strategies_tried': len(self.strategy_performance)
        }

# =============================================================================
# ADVANCED REASONING COMPONENTS
# =============================================================================

class AdvancedEvaluator:
    """
    Advanced evaluator with uncertainty handling and robustness analysis
    """

    def __init__(self):
        self.basic_evaluator = BasicEvaluator()

    def evaluate_with_uncertainty(self, problem: DecisionProblem,
                                 uncertainty_factor: float = 0.1) -> List[Tuple[DecisionOption, float]]:
        """Evaluate options considering uncertainty in criteria values"""
        results = []

        for option in problem.options:
            scores = []
            # Run multiple evaluations with noise
            for _ in range(10):
                noisy_option = self._add_uncertainty(option, uncertainty_factor)
                temp_problem = DecisionProblem(
                    id="temp", title="temp", description="temp",
                    decision_type=problem.decision_type,
                    criteria=problem.criteria,
                    options=[noisy_option]
                )
                ranked = self.basic_evaluator.evaluate_decision(temp_problem)
                if ranked:
                    scores.append(ranked[0].overall_score)

            # Calculate mean and standard deviation
            mean_score = sum(scores) / len(scores) if scores else 0.0
            std_score = math.sqrt(sum((s - mean_score) ** 2 for s in scores) / len(scores)) if scores else 0.0

            option.overall_score = mean_score
            results.append((option, std_score))

        # Sort by mean score
        results.sort(key=lambda x: x[0].overall_score, reverse=True)
        return results

    def _add_uncertainty(self, option: DecisionOption, uncertainty_factor: float) -> DecisionOption:
        """Add noise to option attributes to simulate uncertainty"""
        noisy_attributes = {}
        for key, value in option.attributes.items():
            if isinstance(value, (int, float)):
                noise = random.uniform(-uncertainty_factor, uncertainty_factor) * value
                noisy_attributes[key] = value + noise
            else:
                noisy_attributes[key] = value

        return DecisionOption(
            id=option.id + "_noisy",
            name=option.name,
            description=option.description,
            attributes=noisy_attributes
        )

class ConstraintEngine:
    """
    Handles constraints in decision making
    """

    def __init__(self):
        self.constraints = []

    def add_constraint(self, constraint_func: Callable[[DecisionOption], bool],
                      description: str, severity: str = "hard"):
        """Add a constraint function"""
        self.constraints.append({
            'function': constraint_func,
            'description': description,
            'severity': severity
        })

    def filter_options(self, options: List[DecisionOption]) -> List[DecisionOption]:
        """Filter options based on constraints"""
        valid_options = []

        for option in options:
            violates_hard_constraint = False

            for constraint in self.constraints:
                if not constraint['function'](option):
                    if constraint['severity'] == 'hard':
                        violates_hard_constraint = True
                        print(f"     ❌ Option {option.name} violates: {constraint['description']}")
                        break
                    else:
                        print(f"     ⚠️  Option {option.name} violates soft constraint: {constraint['description']}")

            if not violates_hard_constraint:
                valid_options.append(option)

        return valid_options

class DeliberativeAgent:
    """
    Advanced agent with sophisticated deliberative reasoning capabilities
    """

    def __init__(self, name: str):
        self.name = name
        self.reasoning_engine = ReasoningEngine()
        self.advanced_evaluator = AdvancedEvaluator()
        self.constraint_engine = ConstraintEngine()

        # Decision context and preferences
        self.preferences = {}
        self.risk_tolerance = 0.5  # 0 = risk averse, 1 = risk seeking
        self.decision_style = "balanced"  # analytical, intuitive, balanced

        # Learning and adaptation
        self.experience_base = []
        self.success_metrics = defaultdict(list)

        print(f"🧠 Created deliberative agent: {self.name}")

    def set_preferences(self, preferences: Dict[str, float]):
        """Set agent's preferences for criteria"""
        self.preferences = preferences
        print(f"   📝 Updated preferences: {len(preferences)} criteria")

    def set_constraints(self, constraints: List[Tuple[Callable, str, str]]):
        """Set decision constraints"""
        self.constraint_engine.constraints = []
        for constraint_func, description, severity in constraints:
            self.constraint_engine.add_constraint(constraint_func, description, severity)
        print(f"   🚧 Added {len(constraints)} constraints")

    def deliberate(self, problem: DecisionProblem,
                  consider_uncertainty: bool = True,
                  explain_decision: bool = True) -> Dict[str, Any]:
        """
        Comprehensive deliberative decision making
        """
        print(f"🤔 {self.name} deliberating on: {problem.title}")

        # Step 1: Apply constraints
        valid_options = self.constraint_engine.filter_options(problem.options)
        if not valid_options:
            return {'error': 'No valid options after applying constraints'}

        print(f"   ✅ {len(valid_options)} valid options after constraints")

        # Step 2: Adjust criteria weights based on preferences
        self._adjust_criteria_weights(problem)

        # Step 3: Choose reasoning approach
        reasoning_strategy = self._select_reasoning_approach(problem)

        # Step 4: Evaluate options
        if consider_uncertainty:
            option_results = self.advanced_evaluator.evaluate_with_uncertainty(problem)
            best_option = option_results[0][0] if option_results else None
            uncertainty = option_results[0][1] if option_results else 0.0
        else:
            ranked_options = self.reasoning_engine.solve_problem(problem, reasoning_strategy)
            best_option = ranked_options
            uncertainty = 0.0

        # Step 5: Risk assessment
        risk_assessment = self._assess_risk(best_option, uncertainty)

        # Step 6: Generate explanation
        explanation = None
        if explain_decision and best_option:
            explanation = self._generate_explanation(problem, best_option, reasoning_strategy, risk_assessment)

        # Step 7: Learn from this decision
        decision_context = {
            'problem_type': problem.decision_type.value,
            'strategy_used': reasoning_strategy.value if reasoning_strategy else None,
            'uncertainty': uncertainty,
            'risk_level': risk_assessment['level'],
            'option_chosen': best_option.id if best_option else None
        }
        self.experience_base.append(decision_context)

        return {
            'chosen_option': best_option,
            'reasoning_strategy': reasoning_strategy.value if reasoning_strategy else None,
            'confidence': 1.0 - uncertainty,
            'risk_assessment': risk_assessment,
            'explanation': explanation,
            'alternatives_considered': len(valid_options),
            'decision_context': decision_context
        }

    def _adjust_criteria_weights(self, problem: DecisionProblem):
        """Adjust criteria weights based on agent preferences"""
        for criterion in problem.criteria:
            if criterion.id in self.preferences:
                preference_modifier = self.preferences[criterion.id]
                criterion.weight *= preference_modifier
                print(f"     🎯 Adjusted {criterion.name} weight by {preference_modifier:.2f}x")

        # Renormalize weights
        total_weight = sum(c.weight for c in problem.criteria)
        if total_weight > 0:
            for criterion in problem.criteria:
                criterion.weight /= total_weight

    def _select_reasoning_approach(self, problem: DecisionProblem) -> ReasoningStrategy:
        """Select reasoning approach based on decision style and problem characteristics"""
        if self.decision_style == "analytical":
            return ReasoningStrategy.ANALYTICAL
        elif self.decision_style == "intuitive":
            return ReasoningStrategy.INTUITIVE
        else:  # balanced
            # Use experience to choose
            if len(self.experience_base) > 5:
                # Find similar problems in experience
                similar_problems = [exp for exp in self.experience_base
                                  if exp['problem_type'] == problem.decision_type.value]
                if similar_problems:
                    # Use most common successful strategy
                    strategy_counts = defaultdict(int)
                    for exp in similar_problems[-10:]:  # Last 10 similar problems
                        if exp['strategy_used']:
                            strategy_counts[exp['strategy_used']] += 1

                    if strategy_counts:
                        best_strategy_name = max(strategy_counts, key=strategy_counts.get)
                        return ReasoningStrategy(best_strategy_name)

            # Default strategy selection
            return self.reasoning_engine._choose_strategy(problem)

    def _assess_risk(self, option: Optional[DecisionOption], uncertainty: float) -> Dict[str, Any]:
        """Assess risk level of chosen option"""
        if not option:
            return {'level': 'unknown', 'factors': [], 'recommendation': 'reconsider'}

        risk_factors = []
        risk_score = 0.0

        # Uncertainty risk
        if uncertainty > 0.2:
            risk_factors.append(f"High uncertainty in evaluation ({uncertainty:.2f})")
            risk_score += uncertainty

        # Score risk (low scores are risky)
        if option.overall_score < 0.5:
            risk_factors.append(f"Low overall score ({option.overall_score:.2f})")
            risk_score += (0.5 - option.overall_score)

        # Determine risk level
        if risk_score < 0.2:
            level = "low"
        elif risk_score < 0.5:
            level = "medium"
        else:
            level = "high"

        # Risk tolerance adjustment
        if self.risk_tolerance > 0.7 and level == "medium":
            recommendation = "acceptable"
        elif self.risk_tolerance < 0.3 and level == "medium":
            recommendation = "reconsider"
        else:
            recommendation = "proceed" if level == "low" else "caution"

        return {
            'level': level,
            'score': risk_score,
            'factors': risk_factors,
            'recommendation': recommendation
        }

    def _generate_explanation(self, problem: DecisionProblem, chosen_option: DecisionOption,
                            strategy: ReasoningStrategy, risk_assessment: Dict[str, Any]) -> Dict[str, Any]:
        """Generate human-readable explanation of decision"""

        # Find top contributing criteria
        top_criteria = sorted(problem.criteria, key=lambda c: c.weight, reverse=True)[:3]

        criteria_explanations = []
        for criterion in top_criteria:
            score = chosen_option.scores.get(criterion.id, 0.0)
            weight = criterion.weight
            contribution = score * weight

            criteria_explanations.append({
                'criterion': criterion.name,
                'score': score,
                'weight': weight,
                'contribution': contribution,
                'importance': 'high' if weight > 0.3 else 'medium' if weight > 0.1 else 'low'
            })

        # Strategy explanation
        strategy_reasons = {
            ReasoningStrategy.ANALYTICAL: "Thorough analysis was needed for this important decision",
            ReasoningStrategy.INTUITIVE: "Quick decision was required, used heuristic approach",
            ReasoningStrategy.SATISFICING: "Found first acceptable solution to save time",
            ReasoningStrategy.OPTIMIZING: "Complex problem required optimization approach"
        }

        return {
            'chosen_option': {
                'name': chosen_option.name,
                'overall_score': chosen_option.overall_score,
                'description': chosen_option.description
            },
            'key_factors': criteria_explanations,
            'strategy_explanation': strategy_reasons.get(strategy, "Standard decision process"),
            'risk_assessment': risk_assessment,
            'confidence_level': 'high' if risk_assessment['level'] == 'low' else 'medium' if risk_assessment['level'] == 'medium' else 'low'
        }

    def evaluate_decision_outcome(self, decision_id: str, actual_outcome: float,
                                expected_outcome: float = None):
        """Learn from actual decision outcomes"""
        if self.experience_base:
            # Find the corresponding decision
            for experience in self.experience_base:
                if experience.get('decision_id') == decision_id:
                    experience['actual_outcome'] = actual_outcome
                    experience['expected_outcome'] = expected_outcome or experience.get('predicted_outcome', 0.0)

                    # Calculate success
                    success = actual_outcome >= (expected_outcome or 0.6)
                    experience['success'] = success

                    # Update success metrics for this strategy
                    strategy = experience.get('strategy_used')
                    if strategy:
                        self.success_metrics[strategy].append(success)

                    print(f"   📊 Recorded outcome for decision {decision_id}: {'✅ Success' if success else '❌ Failed'}")
                    break

    def get_decision_insights(self) -> Dict[str, Any]:
        """Get insights from decision-making history"""
        if not self.experience_base:
            return {'message': 'No decision history available'}

        total_decisions = len(self.experience_base)
        decisions_with_outcomes = sum(1 for exp in self.experience_base if 'actual_outcome' in exp)

        # Strategy performance analysis
        strategy_performance = {}
        for strategy, outcomes in self.success_metrics.items():
            if outcomes:
                success_rate = sum(outcomes) / len(outcomes)
                strategy_performance[strategy] = {
                    'success_rate': success_rate,
                    'total_uses': len(outcomes),
                    'confidence': min(1.0, len(outcomes) / 10)  # More confident with more data
                }

        # Risk tolerance effectiveness
        risk_decisions = [exp for exp in self.experience_base if 'risk_level' in exp]
        risk_analysis = defaultdict(list)
        for exp in risk_decisions:
            if 'success' in exp:
                risk_analysis[exp['risk_level']].append(exp['success'])

        risk_effectiveness = {}
        for risk_level, outcomes in risk_analysis.items():
            if outcomes:
                risk_effectiveness[risk_level] = sum(outcomes) / len(outcomes)

        return {
            'total_decisions': total_decisions,
            'decisions_with_outcomes': decisions_with_outcomes,
            'overall_success_rate': sum(exp.get('success', False) for exp in self.experience_base) / max(decisions_with_outcomes, 1),
            'strategy_performance': strategy_performance,
            'risk_effectiveness': risk_effectiveness,
            'experience_depth': min(1.0, total_decisions / 50),  # How much experience the agent has
            'most_used_strategy': max(self.success_metrics.keys(), key=lambda s: len(self.success_metrics[s])) if self.success_metrics else None
        }

class DecisionExplainer:
    """
    Advanced explanation system for decision processes
    """

    def __init__(self):
        self.explanation_templates = {
            'criterion_importance': "'{criterion}' was given {weight:.1%} importance because {reason}",
            'option_strength': "'{option}' scored {score:.2f} on '{criterion}' because {reason}",
            'strategy_choice': "Used {strategy} strategy because {reason}",
            'final_decision': "Chose '{option}' with {confidence:.1%} confidence because {reason}"
        }

    def explain_decision_process(self, problem: DecisionProblem, chosen_option: DecisionOption,
                               strategy: ReasoningStrategy, detailed: bool = True) -> str:
        """Generate comprehensive decision explanation"""

        explanation = []
        explanation.append(f"Decision Analysis: {problem.title}")
        explanation.append("=" * (20 + len(problem.title)))
        explanation.append("")

        # Problem overview
        explanation.append("📋 Problem Overview:")
        explanation.append(f"   Type: {problem.decision_type.value.title()}")
        explanation.append(f"   Options considered: {len(problem.options)}")
        explanation.append(f"   Evaluation criteria: {len(problem.criteria)}")
        explanation.append("")

        # Strategy explanation
        strategy_reasons = {
            ReasoningStrategy.ANALYTICAL: "this is an important decision requiring thorough analysis",
            ReasoningStrategy.INTUITIVE: "quick decision was needed, so used experience-based heuristics",
            ReasoningStrategy.SATISFICING: "efficiency was prioritized, seeking first acceptable solution",
            ReasoningStrategy.OPTIMIZING: "complexity required comprehensive optimization approach"
        }

        explanation.append("🧠 Reasoning Approach:")
        explanation.append(f"   Strategy: {strategy.value.title()}")
        explanation.append(f"   Rationale: {strategy_reasons.get(strategy, 'Standard decision process')}")
        explanation.append("")

        # Criteria analysis
        explanation.append("📊 Evaluation Criteria:")
        sorted_criteria = sorted(problem.criteria, key=lambda c: c.weight, reverse=True)
        for i, criterion in enumerate(sorted_criteria[:5]):  # Top 5 criteria
            importance = "High" if criterion.weight > 0.25 else "Medium" if criterion.weight > 0.1 else "Low"
            explanation.append(f"   {i+1}. {criterion.name}: {criterion.weight:.1%} weight ({importance} importance)")
            if detailed:
                explanation.append(f"      Direction: {criterion.direction}")
                explanation.append(f"      Description: {criterion.description}")
        explanation.append("")

        # Option evaluation
        explanation.append("🔍 Option Analysis:")
        all_options = sorted(problem.options, key=lambda o: o.overall_score, reverse=True)

        for i, option in enumerate(all_options[:3]):  # Top 3 options
            rank = "🥇" if i == 0 else "🥈" if i == 1 else "🥉"
            explanation.append(f"   {rank} {option.name}: {option.overall_score:.3f} overall score")

            if detailed and i < 2:  # Detailed breakdown for top 2
                explanation.append(f"      Description: {option.description}")
                explanation.append("      Key scores:")

                # Show top contributing criteria
                criterion_scores = []
                for criterion in sorted_criteria[:3]:
                    score = option.scores.get(criterion.id, 0.0)
                    contribution = score * criterion.weight
                    criterion_scores.append((criterion.name, score, contribution))

                for crit_name, score, contribution in criterion_scores:
                    explanation.append(f"        {crit_name}: {score:.2f} (contributes {contribution:.3f})")
                explanation.append("")

        # Final decision
        explanation.append("✅ Final Decision:")
        explanation.append(f"   Chosen: {chosen_option.name}")
        explanation.append(f"   Overall Score: {chosen_option.overall_score:.3f}")
        explanation.append(f"   Key Strengths:")

        # Identify top strengths
        top_criteria = sorted(problem.criteria, key=lambda c: c.weight, reverse=True)[:3]
        for criterion in top_criteria:
            score = chosen_option.scores.get(criterion.id, 0.0)
            if score > 0.7:  # Strong performance
                explanation.append(f"     • Excellent {criterion.name} ({score:.2f}/1.0)")
            elif score > 0.5:
                explanation.append(f"     • Good {criterion.name} ({score:.2f}/1.0)")

        explanation.append("")
        explanation.append("=" * 50)

        return "\n".join(explanation)

class DecisionSimulator:
    """
    Simulator for testing decision-making systems
    """

    def __init__(self):
        self.scenarios = []
        self.results = []

    def add_scenario(self, scenario_name: str, problem_generator: Callable[[], DecisionProblem]):
        """Add a test scenario"""
        self.scenarios.append((scenario_name, problem_generator))

    def run_simulation(self, agent: DeliberativeAgent, num_runs: int = 10) -> Dict[str, Any]:
        """Run simulation across all scenarios"""
        print(f"🎮 Running decision simulation with {len(self.scenarios)} scenarios...")

        scenario_results = {}

        for scenario_name, problem_generator in self.scenarios:
            print(f"   Testing scenario: {scenario_name}")

            scenario_outcomes = []
            for run in range(num_runs):
                # Generate problem instance
                problem = problem_generator()

                # Agent makes decision
                decision_result = agent.deliberate(problem, explain_decision=False)

                # Record outcome
                outcome = {
                    'run': run,
                    'chosen_option': decision_result.get('chosen_option'),
                    'confidence': decision_result.get('confidence', 0.0),
                    'strategy_used': decision_result.get('reasoning_strategy'),
                    'risk_level': decision_result.get('risk_assessment', {}).get('level', 'unknown')
                }
                scenario_outcomes.append(outcome)

            # Analyze scenario results
            avg_confidence = sum(o['confidence'] for o in scenario_outcomes) / len(scenario_outcomes)
            strategy_usage = defaultdict(int)
            risk_distribution = defaultdict(int)

            for outcome in scenario_outcomes:
                strategy_usage[outcome['strategy_used']] += 1
                risk_distribution[outcome['risk_level']] += 1

            scenario_results[scenario_name] = {
                'average_confidence': avg_confidence,
                'strategy_usage': dict(strategy_usage),
                'risk_distribution': dict(risk_distribution),
                'total_runs': num_runs
            }

        return scenario_results


In [7]:
# DEMONSTRATION AND VISUALIZATION
# =============================================================================

def create_sample_decision_problem():
    """Create a realistic decision problem for demonstration"""

    problem = DecisionProblem(
        id="laptop_purchase",
        title="Choose a Laptop for Work",
        description="Select the best laptop considering performance, cost, and portability",
        decision_type=DecisionType.TACTICAL
    )

    # Define what matters (criteria)
    criteria = [
        DecisionCriterion(
            id="cost",
            name="cost",
            description="Purchase price",
            weight=0.3,
            direction="minimize"
        ),
        DecisionCriterion(
            id="performance",
            name="performance_score",
            description="CPU/GPU performance rating",
            weight=0.4,
            direction="maximize"
        ),
        DecisionCriterion(
            id="portability",
            name="weight",
            description="Device weight in pounds",
            weight=0.3,
            direction="minimize"
        )
    ]

    for criterion in criteria:
        problem.criteria.append(criterion)

    # Define options
    laptops = [
        DecisionOption(
            id="ultrabook",
            name="Premium Ultrabook",
            description="Lightweight premium laptop",
            attributes={"cost": 1500, "performance_score": 75, "weight": 2.5}
        ),
        DecisionOption(
            id="gaming",
            name="Gaming Laptop",
            description="High-performance gaming machine",
            attributes={"cost": 2000, "performance_score": 95, "weight": 6.0}
        ),
        DecisionOption(
            id="budget",
            name="Budget Laptop",
            description="Basic laptop for everyday tasks",
            attributes={"cost": 800, "performance_score": 50, "weight": 4.0}
        ),
        DecisionOption(
            id="workstation",
            name="Mobile Workstation",
            description="Professional workstation laptop",
            attributes={"cost": 2500, "performance_score": 90, "weight": 5.5}
        )
    ]

    for laptop in laptops:
        problem.options.append(laptop)

    return problem

def create_restaurant_decision_problem():
    """Create a restaurant selection decision problem"""

    problem = DecisionProblem(
        id="restaurant_choice",
        title="Choose Restaurant for Dinner",
        description="Select restaurant considering food quality, price, and atmosphere",
        decision_type=DecisionType.OPERATIONAL
    )

    criteria = [
        DecisionCriterion(id="quality", name="food_quality", description="Food quality rating", weight=0.4, direction="maximize"),
        DecisionCriterion(id="price", name="average_cost", description="Average meal cost", weight=0.3, direction="minimize"),
        DecisionCriterion(id="atmosphere", name="ambiance_score", description="Atmosphere rating", weight=0.3, direction="maximize")
    ]

    options = [
        DecisionOption(id="italian", name="Bella Vista", description="Upscale Italian restaurant",
                      attributes={"food_quality": 90, "average_cost": 45, "ambiance_score": 85}),
        DecisionOption(id="casual", name="The Grill", description="Casual American restaurant",
                      attributes={"food_quality": 75, "average_cost": 25, "ambiance_score": 70}),
        DecisionOption(id="sushi", name="Sakura", description="High-end sushi bar",
                      attributes={"food_quality": 95, "average_cost": 60, "ambiance_score": 80}),
        DecisionOption(id="pizza", name="Tony's Pizza", description="Family-friendly pizza place",
                      attributes={"food_quality": 70, "average_cost": 20, "ambiance_score": 60})
    ]

    problem.criteria.extend(criteria)
    problem.options.extend(options)
    return problem

def create_investment_decision_problem():
    """Create an investment decision problem"""

    problem = DecisionProblem(
        id="investment_choice",
        title="Investment Portfolio Allocation",
        description="Allocate funds across different investment options",
        decision_type=DecisionType.STRATEGIC
    )

    criteria = [
        DecisionCriterion(id="return", name="expected_return", description="Expected annual return", weight=0.4, direction="maximize"),
        DecisionCriterion(id="risk", name="risk_level", description="Investment risk level", weight=0.3, direction="minimize"),
        DecisionCriterion(id="liquidity", name="liquidity_score", description="How easily can be converted to cash", weight=0.3, direction="maximize")
    ]

    options = [
        DecisionOption(id="stocks", name="Stock Market Index", description="Diversified stock portfolio",
                      attributes={"expected_return": 8, "risk_level": 7, "liquidity_score": 9}),
        DecisionOption(id="bonds", name="Government Bonds", description="Safe government securities",
                      attributes={"expected_return": 3, "risk_level": 2, "liquidity_score": 8}),
        DecisionOption(id="realestate", name="Real Estate", description="Property investment",
                      attributes={"expected_return": 6, "risk_level": 5, "liquidity_score": 3}),
        DecisionOption(id="crypto", name="Cryptocurrency", description="Digital currency investment",
                      attributes={"expected_return": 15, "risk_level": 9, "liquidity_score": 7})
    ]

    problem.criteria.extend(criteria)
    problem.options.extend(options)
    return problem

def demonstrate_reasoning_strategies():
    """Show how different reasoning strategies work on the same problem"""

    print("\n🧠 DEMONSTRATION: Different Reasoning Strategies")
    print("=" * 50)
    print("Same problem, different ways of thinking about it...")
    print()

    # Create the problem
    problem = create_sample_decision_problem()
    engine = ReasoningEngine()

    print(f"📋 Problem: {problem.title}")
    print(f"   Criteria: {len(problem.criteria)} factors to consider")
    print(f"   Options: {len(problem.options)} laptops to choose from")
    print()

    # Test each strategy
    strategies = [
        ReasoningStrategy.INTUITIVE,
        ReasoningStrategy.SATISFICING,
        ReasoningStrategy.ANALYTICAL,
        ReasoningStrategy.OPTIMIZING
    ]

    results = {}

    for strategy in strategies:
        print(f"--- {strategy.value.upper()} REASONING ---")

        # Create fresh copy of problem for each strategy
        import copy
        test_problem = copy.deepcopy(problem)

        start_time = time.time()
        selected = engine.solve_problem(test_problem, strategy)
        decision_time = time.time() - start_time

        if selected:
            results[strategy.value] = {
                'choice': selected.name,
                'score': selected.overall_score,
                'time': decision_time
            }
            print(f"   Choice: {selected.name}")
            print(f"   Score: {selected.overall_score:.3f}")
            print(f"   Time: {decision_time:.3f}s")
        else:
            results[strategy.value] = {'choice': 'No decision', 'score': 0.0, 'time': decision_time}
            print(f"   No decision made")

        print()

    # Summary comparison
    print("📊 STRATEGY COMPARISON:")
    print("-" * 30)
    for strategy, result in results.items():
        print(f"{strategy.capitalize():12}: {result['choice']} (score: {result['score']:.3f}, time: {result['time']:.3f}s)")

    print("\n💡 Key Insights:")
    print("• Different strategies can lead to different choices")
    print("• Speed vs. thoroughness trade-offs are real")
    print("• The 'best' strategy depends on your situation")
    print("• Meta-reasoning (choosing how to think) is crucial")

    return results

def demonstrate_agent_learning():
    """Show how an agent learns and improves over multiple decisions"""

    print("\n🤖 DEMONSTRATION: Agent Learning Over Time")
    print("=" * 45)
    print("Watch how an agent gets better at making decisions...")
    print()

    # Create learning agent
    agent = LearningAgent("LearnBot")

    # Create several similar problems for learning
    problems = []

    for i in range(5):
        problem = DecisionProblem(
            id=f"decision_{i}",
            title=f"Software Tool Selection {i+1}",
            description=f"Choose software tool for project {i+1}",
            decision_type=DecisionType.TACTICAL
        )

        # Consistent criteria across problems
        criteria = [
            DecisionCriterion(id="cost", name="cost", weight=0.4, direction="minimize"),
            DecisionCriterion(id="features", name="feature_count", weight=0.3, direction="maximize"),
            DecisionCriterion(id="ease", name="ease_of_use", weight=0.3, direction="maximize")
        ]
        for c in criteria:
            problem.criteria.append(c)

        # Different options each time
        options = [
            DecisionOption(f"tool_a_{i}", f"Tool A{i+1}", "Basic tool",
                          {"cost": 100 + i*20, "feature_count": 50 + i*5, "ease_of_use": 80}),
            DecisionOption(f"tool_b_{i}", f"Tool B{i+1}", "Advanced tool",
                          {"cost": 300 + i*30, "feature_count": 90 + i*2, "ease_of_use": 60}),
            DecisionOption(f"tool_c_{i}", f"Tool C{i+1}", "Premium tool",
                          {"cost": 500 + i*50, "feature_count": 95 + i, "ease_of_use": 90})
        ]
        for opt in options:
            problem.options.append(opt)

        problems.append(problem)

    # Watch the agent learn
    print("📚 Learning Progress:")
    print("-" * 20)

    for i, problem in enumerate(problems):
        print(f"\nDecision {i+1}: {problem.title}")

        # Show learning state before decision
        if i > 0:
            summary = agent.get_learning_summary()
            print(f"   📈 Current success rate: {summary['overall_success_rate']:.1%}")
            if summary['best_strategy']:
                print(f"   🧠 Preferred strategy: {summary['best_strategy']}")

        # Make decision
        result = agent.make_decision(problem)

        if result:
            print(f"   ✅ Chose: {result.name} (score: {result.overall_score:.3f})")
        else:
            print(f"   ❌ Failed to decide")

    # Final learning summary
    final_summary = agent.get_learning_summary()
    print(f"\n🎓 FINAL LEARNING SUMMARY:")
    print(f"   Total decisions made: {final_summary['total_decisions']}")
    print(f"   Overall success rate: {final_summary['overall_success_rate']:.1%}")
    print(f"   Best strategy learned: {final_summary['best_strategy']}")
    print(f"   Patterns recognized: {final_summary['patterns_learned']}")

    return final_summary

def demonstrate_deliberative_agent():
    """Demonstrate advanced deliberative reasoning capabilities"""

    print("\n🧠 DEMONSTRATION: Advanced Deliberative Agent")
    print("=" * 50)
    print()

    # Create deliberative agent
    agent = DeliberativeAgent("DeliberativeBot")

    # Set agent preferences and constraints
    agent.set_preferences({
        "cost": 0.8,  # Cost-conscious
        "performance": 1.2,  # Performance-oriented
        "portability": 1.0
    })

    # Add constraints
    constraints = [
        (lambda opt: opt.attributes.get("cost", 0) <= 2000, "Budget constraint: max $2000", "hard"),
        (lambda opt: opt.attributes.get("performance_score", 0) >= 70, "Minimum performance requirement", "soft")
    ]
    agent.set_constraints(constraints)

    # Test on different problem types
    problems = [
        ("Laptop Purchase", create_sample_decision_problem()),
        ("Restaurant Choice", create_restaurant_decision_problem()),
        ("Investment Decision", create_investment_decision_problem())
    ]

    for problem_name, problem in problems:
        print(f"🤔 Deliberating on: {problem_name}")

        # Make decision with full deliberation
        result = agent.deliberate(problem, consider_uncertainty=True, explain_decision=True)

        if 'error' in result:
            print(f"   ❌ {result['error']}")
            continue

        chosen = result['chosen_option']
        confidence = result['confidence']
        strategy = result['reasoning_strategy']
        risk = result['risk_assessment']

        print(f"   ✅ Chose: {chosen.name}")
        print(f"   📊 Score: {chosen.overall_score:.3f}")
        print(f"   🎯 Strategy: {strategy}")
        print(f"   💯 Confidence: {confidence:.1%}")
        print(f"   ⚠️  Risk: {risk['level']} ({risk['recommendation']})")

        # Show explanation highlights
        if result['explanation']:
            explanation = result['explanation']
            print(f"   🔍 Key factors:")
            for factor in explanation['key_factors'][:2]:
                print(f"     • {factor['criterion']}: {factor['score']:.2f} (weight: {factor['weight']:.1%})")

        print()

    # Show learning insights
    insights = agent.get_decision_insights()
    print("📈 AGENT LEARNING INSIGHTS:")
    print(f"   Decisions made: {insights['total_decisions']}")
    print(f"   Most used strategy: {insights.get('most_used_strategy', 'None yet')}")
    if insights['strategy_performance']:
        best_strategy = max(insights['strategy_performance'].items(),
                          key=lambda x: x[1]['success_rate'])
        print(f"   Best performing strategy: {best_strategy[0]} ({best_strategy[1]['success_rate']:.1%} success)")

    return agent

def demonstrate_decision_explanation():
    """Show comprehensive decision explanation capabilities"""

    print("\n🔍 DEMONSTRATION: Decision Explanation System")
    print("=" * 50)
    print()

    # Create explainer and decision components
    explainer = DecisionExplainer()
    problem = create_sample_decision_problem()
    engine = ReasoningEngine()

    # Make a decision
    chosen_option = engine.solve_problem(problem, ReasoningStrategy.ANALYTICAL)

    if chosen_option:
        # Generate comprehensive explanation
        explanation = explainer.explain_decision_process(
            problem, chosen_option, ReasoningStrategy.ANALYTICAL, detailed=True
        )

        print(explanation)
    else:
        print("❌ No decision made to explain")

def demonstrate_simulation_testing():
    """Demonstrate decision simulation and testing"""

    print("\n🎮 DEMONSTRATION: Decision Simulation Testing")
    print("=" * 50)
    print()

    # Create simulator
    simulator = DecisionSimulator()

    # Add test scenarios
    simulator.add_scenario("laptop_selection", create_sample_decision_problem)
    simulator.add_scenario("restaurant_choice", create_restaurant_decision_problem)
    simulator.add_scenario("investment_allocation", create_investment_decision_problem)

    # Create agent for testing
    test_agent = DeliberativeAgent("TestAgent")
    test_agent.risk_tolerance = 0.7  # Risk-tolerant
    test_agent.decision_style = "analytical"

    # Run simulation
    results = simulator.run_simulation(test_agent, num_runs=5)

    print("📊 SIMULATION RESULTS:")
    print("-" * 30)

    for scenario, result in results.items():
        print(f"\n{scenario.replace('_', ' ').title()}:")
        print(f"   Average confidence: {result['average_confidence']:.1%}")
        print(f"   Strategy usage:")
        for strategy, count in result['strategy_usage'].items():
            percentage = (count / result['total_runs']) * 100
            print(f"     {strategy}: {percentage:.0f}%")
        print(f"   Risk distribution:")
        for risk, count in result['risk_distribution'].items():
            percentage = (count / result['total_runs']) * 100
            print(f"     {risk}: {percentage:.0f}%")

# MAIN DEMONSTRATION SEQUENCE
# =============================================================================

print("🔧 Tutorial 11 initialization complete!")
print("✅ All classes loaded successfully:")
print("   - DecisionProblem for structuring choices")
print("   - DecisionCriterion for evaluation dimensions")
print("   - BasicEvaluator for multi-criteria analysis")
print("   - ReasoningEngine for strategy selection")
print("   - LearningAgent for experience-based improvement")
print("   - DeliberativeAgent for advanced reasoning")
print("   - DecisionExplainer for transparency")
print("   - DecisionSimulator for testing")
print()
print("🚀 Ready to build intelligent decision-making systems!")
print()

🔧 Tutorial 11 initialization complete!
✅ All classes loaded successfully:
   - DecisionProblem for structuring choices
   - DecisionCriterion for evaluation dimensions
   - BasicEvaluator for multi-criteria analysis
   - ReasoningEngine for strategy selection
   - LearningAgent for experience-based improvement
   - DeliberativeAgent for advanced reasoning
   - DecisionExplainer for transparency
   - DecisionSimulator for testing

🚀 Ready to build intelligent decision-making systems!



In [9]:
# DEMO SECTION: Let's build sophisticated decision systems!
# =============================================================================

print("=" * 60)
print("🚀 Tutorial 11: Deliberative Reasoning - Smart Decision Making")
print("=" * 60)
print()

🚀 Tutorial 11: Deliberative Reasoning - Smart Decision Making



In [10]:
# Step 1: Basic decision-making demonstration
print("📝 Step 1: Basic Multi-Criteria Decision Making...")

# Create a sample decision problem
problem = create_sample_decision_problem()
print(f"   📋 Created decision problem: {problem.title}")
print(f"   📊 {len(problem.criteria)} criteria, {len(problem.options)} options")

# Basic evaluation
evaluator = BasicEvaluator()
ranked_options = evaluator.evaluate_decision(problem)

print(f"   🏆 Top choice: {ranked_options[0].name} (score: {ranked_options[0].overall_score:.3f})")
print(f"   📈 All rankings:")
for i, option in enumerate(ranked_options):
    print(f"      {i+1}. {option.name}: {option.overall_score:.3f}")

print()

📝 Step 1: Basic Multi-Criteria Decision Making...
   📋 Created decision problem: Choose a Laptop for Work
   📊 3 criteria, 4 options
   📊 Evaluating 4 options against 3 criteria...
   ✅ Evaluation complete
   🏆 Top choice: Gaming Laptop (score: 0.974)
   📈 All rankings:
      1. Gaming Laptop: 0.974
      2. Mobile Workstation: 0.952
      3. Premium Ultrabook: 0.895
      4. Budget Laptop: 0.798



In [11]:
# Step 2: Reasoning strategy comparison
print("📝 Step 2: Comparing Different Reasoning Strategies...")
strategy_results = demonstrate_reasoning_strategies()
print()

📝 Step 2: Comparing Different Reasoning Strategies...

🧠 DEMONSTRATION: Different Reasoning Strategies
Same problem, different ways of thinking about it...

📋 Problem: Choose a Laptop for Work
   Criteria: 3 factors to consider
   Options: 4 laptops to choose from

--- INTUITIVE REASONING ---
   🧠 Using intuitive reasoning...
     ⚡ Quick heuristic evaluation...
   Choice: Gaming Laptop
   Score: 0.000
   Time: 0.000s

--- SATISFICING REASONING ---
   🧠 Using satisficing reasoning...
     ✋ Looking for first acceptable option...
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
     ✅ Found acceptable option: Premium Ultrabook
   Choice: Premium Ultrabook
   Score: 0.895
   Time: 0.000s

--- ANALYTICAL REASONING ---
   🧠 Using analytical reasoning...
     📋 Systematic evaluation of all options...
   📊 Evaluating 4 options against 3 criteria...
   ✅ Evaluation complete
     🔍 Checking robustness of decision...
     ⚠️  Top options are very close - decision is sens

In [13]:
# Step 3: Learning agent demonstration


In [14]:








# Step 4: Advanced deliberative reasoning
print("📝 Step 4: Advanced Deliberative Reasoning...")
deliberative_agent = demonstrate_deliberative_agent()
print()


📝 Step 4: Advanced Deliberative Reasoning...

🧠 DEMONSTRATION: Advanced Deliberative Agent

🧠 Created deliberative agent: DeliberativeBot
   📝 Updated preferences: 3 criteria
   🚧 Added 2 constraints
🤔 Deliberating on: Laptop Purchase
🤔 DeliberativeBot deliberating on: Choose a Laptop for Work
     ⚠️  Option Budget Laptop violates soft constraint: Minimum performance requirement
     ❌ Option Mobile Workstation violates: Budget constraint: max $2000
   ✅ 3 valid options after constraints
     🎯 Adjusted cost weight by 0.80x
     🎯 Adjusted performance_score weight by 1.20x
     🎯 Adjusted weight weight by 1.00x
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating

In [15]:
# Fix the demonstrate_agent_learning function
def demonstrate_agent_learning():
    """Show how an agent learns and improves over multiple decisions"""

    print("\n🤖 DEMONSTRATION: Agent Learning Over Time")
    print("=" * 45)
    print("Watch how an agent gets better at making decisions...")
    print()

    # Create learning agent
    agent = LearningAgent("LearnBot")

    # Create several similar problems for learning
    problems = []

    for i in range(5):
        problem = DecisionProblem(
            id=f"decision_{i}",
            title=f"Software Tool Selection {i+1}",
            description=f"Choose software tool for project {i+1}",
            decision_type=DecisionType.TACTICAL
        )

        # Fixed: Added missing descriptions
        criteria = [
            DecisionCriterion(id="cost", name="cost", description="Software cost", weight=0.4, direction="minimize"),
            DecisionCriterion(id="features", name="feature_count", description="Number of features", weight=0.3, direction="maximize"),
            DecisionCriterion(id="ease", name="ease_of_use", description="Ease of use rating", weight=0.3, direction="maximize")
        ]
        for c in criteria:
            problem.criteria.append(c)

        # Different options each time
        options = [
            DecisionOption(f"tool_a_{i}", f"Tool A{i+1}", "Basic tool",
                          {"cost": 100 + i*20, "feature_count": 50 + i*5, "ease_of_use": 80}),
            DecisionOption(f"tool_b_{i}", f"Tool B{i+1}", "Advanced tool",
                          {"cost": 300 + i*30, "feature_count": 90 + i*2, "ease_of_use": 60}),
            DecisionOption(f"tool_c_{i}", f"Tool C{i+1}", "Premium tool",
                          {"cost": 500 + i*50, "feature_count": 95 + i, "ease_of_use": 90})
        ]
        for opt in options:
            problem.options.append(opt)

        problems.append(problem)

    # Watch the agent learn
    print("📚 Learning Progress:")
    print("-" * 20)

    for i, problem in enumerate(problems):
        print(f"\nDecision {i+1}: {problem.title}")

        # Show learning state before decision
        if i > 0:
            summary = agent.get_learning_summary()
            print(f"   📈 Current success rate: {summary['overall_success_rate']:.1%}")
            if summary['best_strategy']:
                print(f"   🧠 Preferred strategy: {summary['best_strategy']}")

        # Make decision
        result = agent.make_decision(problem)

        if result:
            print(f"   ✅ Chose: {result.name} (score: {result.overall_score:.3f})")
        else:
            print(f"   ❌ Failed to decide")

    # Final learning summary
    final_summary = agent.get_learning_summary()
    print(f"\n🎓 FINAL LEARNING SUMMARY:")
    print(f"   Total decisions made: {final_summary['total_decisions']}")
    print(f"   Overall success rate: {final_summary['overall_success_rate']:.1%}")
    print(f"   Best strategy learned: {final_summary['best_strategy']}")
    print(f"   Patterns recognized: {final_summary['patterns_learned']}")

    return final_summary

# Now run step 3
print("📝 Step 3: Agent Learning from Experience...")
learning_summary = demonstrate_agent_learning()
print()

📝 Step 3: Agent Learning from Experience...

🤖 DEMONSTRATION: Agent Learning Over Time
Watch how an agent gets better at making decisions...

🤖 Created learning agent: LearnBot
📚 Learning Progress:
--------------------

Decision 1: Software Tool Selection 1
🤖 LearnBot deciding on: Software Tool Selection 1
   🧠 Using satisficing reasoning...
     ✋ Looking for first acceptable option...
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
     ✅ Found acceptable option: Tool A1
   ✅ Chose: Tool A1 (score: 0.790)

Decision 2: Software Tool Selection 2
   📈 Current success rate: 100.0%
   🧠 Preferred strategy: satisficing
🤖 LearnBot deciding on: Software Tool Selection 2
   📈 Using best-performing strategy: satisficing
   🧠 Using satisficing reasoning...
     ✋ Looking for first acceptable option...
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
     ✅ Found acceptable option: Tool A2
   ✅ Chose: Tool A2 (score: 0.805)

Decision 3: Software T

In [16]:
# Step 4: Advanced deliberative reasoning
print("📝 Step 4: Advanced Deliberative Reasoning...")
deliberative_agent = demonstrate_deliberative_agent()
print()

📝 Step 4: Advanced Deliberative Reasoning...

🧠 DEMONSTRATION: Advanced Deliberative Agent

🧠 Created deliberative agent: DeliberativeBot
   📝 Updated preferences: 3 criteria
   🚧 Added 2 constraints
🤔 Deliberating on: Laptop Purchase
🤔 DeliberativeBot deliberating on: Choose a Laptop for Work
     ⚠️  Option Budget Laptop violates soft constraint: Minimum performance requirement
     ❌ Option Mobile Workstation violates: Budget constraint: max $2000
   ✅ 3 valid options after constraints
     🎯 Adjusted cost weight by 0.80x
     🎯 Adjusted performance_score weight by 1.20x
     🎯 Adjusted weight weight by 1.00x
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating

In [17]:
# Step 5: Decision explanation system
print("📝 Step 5: Decision Explanation and Transparency...")
demonstrate_decision_explanation()
print()

📝 Step 5: Decision Explanation and Transparency...

🔍 DEMONSTRATION: Decision Explanation System

   🧠 Using analytical reasoning...
     📋 Systematic evaluation of all options...
   📊 Evaluating 4 options against 3 criteria...
   ✅ Evaluation complete
     🔍 Checking robustness of decision...
     ⚠️  Top options are very close - decision is sensitive
Decision Analysis: Choose a Laptop for Work

📋 Problem Overview:
   Type: Tactical
   Options considered: 4
   Evaluation criteria: 3

🧠 Reasoning Approach:
   Strategy: Analytical
   Rationale: this is an important decision requiring thorough analysis

📊 Evaluation Criteria:
   1. performance_score: 40.0% weight (High importance)
      Direction: maximize
      Description: CPU/GPU performance rating
   2. cost: 30.0% weight (High importance)
      Direction: minimize
      Description: Purchase price
   3. weight: 30.0% weight (High importance)
      Direction: minimize
      Description: Device weight in pounds

🔍 Option Analysis:
   

In [18]:
# Step 6: Constraint handling demonstration
print("📝 Step 6: Constraint Handling and Filtering...")

constraint_engine = ConstraintEngine()

# Add various constraints
constraint_engine.add_constraint(
    lambda opt: opt.attributes.get("cost", 0) <= 1800,
    "Budget must be under $1800",
    "hard"
)

constraint_engine.add_constraint(
    lambda opt: opt.attributes.get("weight", 10) <= 5.0,
    "Weight should be under 5 pounds",
    "soft"
)

# Test constraint filtering
test_problem = create_sample_decision_problem()
print(f"   Original options: {len(test_problem.options)}")

valid_options = constraint_engine.filter_options(test_problem.options)
print(f"   Valid options after constraints: {len(valid_options)}")

for option in valid_options:
    cost = option.attributes.get("cost", 0)
    weight = option.attributes.get("weight", 0)
    print(f"     ✅ {option.name}: ${cost}, {weight}lbs")

print()


📝 Step 6: Constraint Handling and Filtering...
   Original options: 4
     ❌ Option Gaming Laptop violates: Budget must be under $1800
     ❌ Option Mobile Workstation violates: Budget must be under $1800
   Valid options after constraints: 2
     ✅ Premium Ultrabook: $1500, 2.5lbs
     ✅ Budget Laptop: $800, 4.0lbs



In [19]:
# Step 7: Uncertainty handling
print("📝 Step 7: Handling Uncertainty in Evaluations...")

advanced_evaluator = AdvancedEvaluator()
uncertain_results = advanced_evaluator.evaluate_with_uncertainty(test_problem, uncertainty_factor=0.15)

print("   🎲 Results with uncertainty consideration:")
for option, uncertainty in uncertain_results[:3]:
    print(f"      {option.name}: {option.overall_score:.3f} ± {uncertainty:.3f}")

print()

📝 Step 7: Handling Uncertainty in Evaluations...
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
  

In [20]:
# Step 8: Comprehensive decision simulation
print("📝 Step 8: Decision System Testing and Simulation...")
demonstrate_simulation_testing()
print()


📝 Step 8: Decision System Testing and Simulation...

🎮 DEMONSTRATION: Decision Simulation Testing

🧠 Created deliberative agent: TestAgent
🎮 Running decision simulation with 3 scenarios...
   Testing scenario: laptop_selection
🤔 TestAgent deliberating on: Choose a Laptop for Work
   ✅ 4 valid options after constraints
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 optio

In [21]:
# Step 9: Real-world scenario testing
print("📝 Step 9: Real-World Decision Scenarios...")

# Create complex real-world scenarios
scenarios = [
    {
        'name': 'Emergency Response',
        'type': DecisionType.EMERGENCY,
        'problem': DecisionProblem(
            id="emergency_response",
            title="Emergency System Response",
            description="Choose immediate response to system outage",
            decision_type=DecisionType.EMERGENCY,
            criteria=[
                DecisionCriterion("speed", "response_time", "Time to implement", 0.5, "minimize"),
                DecisionCriterion("effectiveness", "fix_probability", "Likelihood of success", 0.3, "maximize"),
                DecisionCriterion("cost", "resource_cost", "Resource requirements", 0.2, "minimize")
            ],
            options=[
                DecisionOption("restart", "System Restart", "Quick restart",
                              {"response_time": 2, "fix_probability": 70, "resource_cost": 10}),
                DecisionOption("rollback", "Version Rollback", "Revert to previous version",
                              {"response_time": 10, "fix_probability": 90, "resource_cost": 30}),
                DecisionOption("hotfix", "Emergency Hotfix", "Deploy quick fix",
                              {"response_time": 30, "fix_probability": 80, "resource_cost": 50}),
                DecisionOption("maintenance", "Full Maintenance", "Complete system check",
                              {"response_time": 120, "fix_probability": 95, "resource_cost": 100})
            ]
        )
    },
    {
        'name': 'Strategic Planning',
        'type': DecisionType.STRATEGIC,
        'problem': DecisionProblem(
            id="strategic_expansion",
            title="Market Expansion Strategy",
            description="Choose market expansion approach",
            decision_type=DecisionType.STRATEGIC,
            criteria=[
                DecisionCriterion("roi", "expected_roi", "Return on investment", 0.3, "maximize"),
                DecisionCriterion("risk", "risk_level", "Business risk", 0.25, "minimize"),
                DecisionCriterion("timeline", "time_to_market", "Implementation speed", 0.2, "minimize"),
                DecisionCriterion("scalability", "growth_potential", "Long-term growth", 0.25, "maximize")
            ],
            options=[
                DecisionOption("organic", "Organic Growth", "Gradual internal expansion",
                              {"expected_roi": 15, "risk_level": 3, "time_to_market": 24, "growth_potential": 70}),
                DecisionOption("acquisition", "Strategic Acquisition", "Buy existing company",
                              {"expected_roi": 25, "risk_level": 7, "time_to_market": 6, "growth_potential": 85}),
                DecisionOption("partnership", "Joint Venture", "Partner with local company",
                              {"expected_roi": 20, "risk_level": 5, "time_to_market": 12, "growth_potential": 75}),
                DecisionOption("franchise", "Franchise Model", "License business model",
                              {"expected_roi": 18, "risk_level": 4, "time_to_market": 8, "growth_potential": 80})
            ]
        )
    }
]

# Test different agents on scenarios
agents = [
    ("Conservative Agent", DeliberativeAgent("ConservativeBot")),
    ("Aggressive Agent", DeliberativeAgent("AggressiveBot")),
    ("Balanced Agent", DeliberativeAgent("BalancedBot"))
]

# Configure agents differently
agents[0][1].risk_tolerance = 0.2  # Conservative
agents[0][1].decision_style = "analytical"

agents[1][1].risk_tolerance = 0.8  # Aggressive
agents[1][1].decision_style = "optimizing"

agents[2][1].risk_tolerance = 0.5  # Balanced
agents[2][1].decision_style = "balanced"

# Test scenarios
for scenario in scenarios:
    print(f"\n🎯 Scenario: {scenario['name']}")
    print(f"   Type: {scenario['type'].value}")
    print(f"   Problem: {scenario['problem'].description}")

    for agent_name, agent in agents:
        result = agent.deliberate(scenario['problem'], explain_decision=False)

        if 'error' not in result:
            chosen = result['chosen_option']
            print(f"   {agent_name}: {chosen.name} (confidence: {result['confidence']:.1%})")
        else:
            print(f"   {agent_name}: Failed to decide")

print()


📝 Step 9: Real-World Decision Scenarios...
🧠 Created deliberative agent: ConservativeBot
🧠 Created deliberative agent: AggressiveBot
🧠 Created deliberative agent: BalancedBot

🎯 Scenario: Emergency Response
   Type: emergency
   Problem: Choose immediate response to system outage
🤔 ConservativeBot deliberating on: Emergency System Response
   ✅ 4 valid options after constraints
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3

In [22]:
# Step 10: Performance analysis and optimization
print("📝 Step 10: Performance Analysis and Decision Quality...")

# Create performance analyzer
class DecisionPerformanceAnalyzer:
    def __init__(self):
        self.decision_records = []

    def record_decision(self, problem: DecisionProblem, chosen_option: DecisionOption,
                       strategy: ReasoningStrategy, decision_time: float, agent_name: str):
        record = {
            'problem_id': problem.id,
            'problem_type': problem.decision_type.value,
            'chosen_option': chosen_option.name,
            'score': chosen_option.overall_score,
            'strategy': strategy.value,
            'decision_time': decision_time,
            'agent_name': agent_name,
            'criteria_count': len(problem.criteria),
            'options_count': len(problem.options)
        }
        self.decision_records.append(record)

    def analyze_performance(self) -> Dict[str, Any]:
        if not self.decision_records:
            return {'error': 'No decision records to analyze'}

        # Overall statistics
        total_decisions = len(self.decision_records)
        avg_score = sum(r['score'] for r in self.decision_records) / total_decisions
        avg_time = sum(r['decision_time'] for r in self.decision_records) / total_decisions

        # Strategy performance
        strategy_stats = defaultdict(lambda: {'count': 0, 'total_score': 0.0, 'total_time': 0.0})
        for record in self.decision_records:
            strategy = record['strategy']
            strategy_stats[strategy]['count'] += 1
            strategy_stats[strategy]['total_score'] += record['score']
            strategy_stats[strategy]['total_time'] += record['decision_time']

        strategy_performance = {}
        for strategy, stats in strategy_stats.items():
            strategy_performance[strategy] = {
                'avg_score': stats['total_score'] / stats['count'],
                'avg_time': stats['total_time'] / stats['count'],
                'usage_count': stats['count']
            }

        # Problem complexity analysis
        complexity_stats = defaultdict(list)
        for record in self.decision_records:
            complexity = record['criteria_count'] * record['options_count']
            complexity_stats[complexity].append(record)

        return {
            'total_decisions': total_decisions,
            'average_score': avg_score,
            'average_time': avg_time,
            'strategy_performance': strategy_performance,
            'complexity_analysis': {
                'simple_problems': len([r for r in self.decision_records if r['criteria_count'] * r['options_count'] <= 12]),
                'complex_problems': len([r for r in self.decision_records if r['criteria_count'] * r['options_count'] > 12])
            }
        }

# Run performance analysis
analyzer = DecisionPerformanceAnalyzer()

# Generate test decisions for analysis
test_problems = [
    create_sample_decision_problem(),
    create_restaurant_decision_problem(),
    create_investment_decision_problem()
]

test_agent = DeliberativeAgent("PerformanceTestAgent")

print("   🔄 Running performance test decisions...")
for i, problem in enumerate(test_problems * 3):  # Run each problem 3 times
    start_time = time.time()

    # Use different strategies
    strategies = [ReasoningStrategy.ANALYTICAL, ReasoningStrategy.INTUITIVE, ReasoningStrategy.OPTIMIZING]
    strategy = strategies[i % len(strategies)]

    chosen = test_agent.reasoning_engine.solve_problem(problem, strategy)
    decision_time = time.time() - start_time

    if chosen:
        analyzer.record_decision(problem, chosen, strategy, decision_time, test_agent.name)

# Analyze results
performance_results = analyzer.analyze_performance()
print("   📊 Performance Analysis Results:")
print(f"      Total decisions analyzed: {performance_results['total_decisions']}")
print(f"      Average decision score: {performance_results['average_score']:.3f}")
print(f"      Average decision time: {performance_results['average_time']:.3f}s")

print("   📈 Strategy Performance:")
for strategy, stats in performance_results['strategy_performance'].items():
    print(f"      {strategy}: avg score {stats['avg_score']:.3f}, avg time {stats['avg_time']:.3f}s")

print()


📝 Step 10: Performance Analysis and Decision Quality...
🧠 Created deliberative agent: PerformanceTestAgent
   🔄 Running performance test decisions...
   🧠 Using analytical reasoning...
     📋 Systematic evaluation of all options...
   📊 Evaluating 4 options against 3 criteria...
   ✅ Evaluation complete
     🔍 Checking robustness of decision...
     ⚠️  Top options are very close - decision is sensitive
   🧠 Using intuitive reasoning...
     ⚡ Quick heuristic evaluation...
   🧠 Using optimizing reasoning...
     🎯 Optimizing: trying multiple evaluation approaches...
   📊 Evaluating 4 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 4 options against 3 criteria...
   ✅ Evaluation complete
     🤝 Finding consensus across methods...
     ✅ Both methods agree!
   🧠 Using analytical reasoning...
     📋 Systematic evaluation of all options...
   📊 Evaluating 4 options against 3 criteria...
   ✅ Evaluation complete
     🔍 Checking robustness of decision...
     ⚠️  Top o

In [23]:
# Step 11: Advanced constraint and preference modeling
print("📝 Step 11: Advanced Constraint and Preference Modeling...")

class AdvancedConstraintEngine:
    def __init__(self):
        self.hard_constraints = []
        self.soft_constraints = []
        self.preference_functions = []

    def add_hard_constraint(self, constraint_func: Callable, description: str):
        self.hard_constraints.append((constraint_func, description))

    def add_soft_constraint(self, constraint_func: Callable, description: str, penalty: float = 0.1):
        self.soft_constraints.append((constraint_func, description, penalty))

    def add_preference_function(self, preference_func: Callable, description: str, weight: float = 1.0):
        self.preference_functions.append((preference_func, description, weight))

    def evaluate_option(self, option: DecisionOption) -> Tuple[bool, float, List[str]]:
        """Evaluate option against all constraints and preferences"""

        # Check hard constraints
        violated_constraints = []
        for constraint_func, description in self.hard_constraints:
            if not constraint_func(option):
                violated_constraints.append(description)

        if violated_constraints:
            return False, 0.0, violated_constraints

        # Calculate soft constraint penalties
        penalty_score = 0.0
        for constraint_func, description, penalty in self.soft_constraints:
            if not constraint_func(option):
                penalty_score += penalty
                violated_constraints.append(f"SOFT: {description}")

        # Calculate preference bonuses
        preference_score = 0.0
        total_preference_weight = 0.0

        for preference_func, description, weight in self.preference_functions:
            try:
                pref_value = preference_func(option)
                preference_score += pref_value * weight
                total_preference_weight += weight
            except:
                pass

        # Normalize preference score
        normalized_preference = preference_score / max(total_preference_weight, 1.0)

        # Final score = base score - penalties + preferences
        final_score = max(0.0, 1.0 - penalty_score + normalized_preference * 0.2)

        return True, final_score, violated_constraints

# Test advanced constraint system
advanced_constraint_engine = AdvancedConstraintEngine()

# Add hard constraints
advanced_constraint_engine.add_hard_constraint(
    lambda opt: opt.attributes.get("cost", 0) <= 2000,
    "Must be within budget"
)

# Add soft constraints
advanced_constraint_engine.add_soft_constraint(
    lambda opt: opt.attributes.get("weight", 10) <= 4.0,
    "Prefer lightweight options",
    penalty=0.15
)

# Add preference functions
advanced_constraint_engine.add_preference_function(
    lambda opt: 1.0 if opt.attributes.get("performance_score", 0) > 80 else 0.0,
    "Prefer high performance",
    weight=0.8
)

advanced_constraint_engine.add_preference_function(
    lambda opt: 1.0 if "premium" in opt.name.lower() else 0.0,
    "Prefer premium options",
    weight=0.3
)

# Test on laptop options
test_problem = create_sample_decision_problem()
print("   🧪 Testing advanced constraint system:")

for option in test_problem.options:
    valid, score, violations = advanced_constraint_engine.evaluate_option(option)
    status = "✅ VALID" if valid else "❌ INVALID"
    print(f"      {option.name}: {status} (score: {score:.3f})")
    if violations:
        for violation in violations[:2]:  # Show first 2 violations
            print(f"        - {violation}")

print()


📝 Step 11: Advanced Constraint and Preference Modeling...
   🧪 Testing advanced constraint system:
      Premium Ultrabook: ✅ VALID (score: 1.055)
      Gaming Laptop: ✅ VALID (score: 0.995)
        - SOFT: Prefer lightweight options
      Budget Laptop: ✅ VALID (score: 1.000)
      Mobile Workstation: ❌ INVALID (score: 0.000)
        - Must be within budget



In [24]:
# Step 12: Decision tree and workflow integration
print("📝 Step 12: Decision Trees and Complex Workflows...")

class DecisionNode:
    def __init__(self, decision_id: str, problem: DecisionProblem):
        self.decision_id = decision_id
        self.problem = problem
        self.children = {}  # outcome -> next DecisionNode
        self.chosen_option = None
        self.completion_callbacks = []

    def add_child(self, outcome: str, child_node: 'DecisionNode'):
        self.children[outcome] = child_node

    def add_completion_callback(self, callback: Callable):
        self.completion_callbacks.append(callback)

class DecisionWorkflow:
    def __init__(self, name: str):
        self.name = name
        self.root_node = None
        self.current_node = None
        self.decision_history = []
        self.workflow_context = {}

    def set_root(self, root_node: DecisionNode):
        self.root_node = root_node
        self.current_node = root_node

    def execute_workflow(self, agent: DeliberativeAgent) -> Dict[str, Any]:
        """Execute the decision workflow"""
        print(f"   🔄 Executing workflow: {self.name}")

        while self.current_node:
            print(f"      📋 Current decision: {self.current_node.problem.title}")

            # Make decision at current node
            result = agent.deliberate(self.current_node.problem, explain_decision=False)

            if 'error' in result:
                return {'error': f"Failed at node {self.current_node.decision_id}"}

            chosen_option = result['chosen_option']
            self.current_node.chosen_option = chosen_option

            # Record decision
            decision_record = {
                'node_id': self.current_node.decision_id,
                'chosen_option': chosen_option.name,
                'score': chosen_option.overall_score,
                'context': self.workflow_context.copy()
            }
            self.decision_history.append(decision_record)

            print(f"         ✅ Chose: {chosen_option.name}")

            # Execute completion callbacks
            for callback in self.current_node.completion_callbacks:
                callback(self.workflow_context, chosen_option)

            # Determine next node based on outcome
            next_node = None
            if chosen_option.id in self.current_node.children:
                next_node = self.current_node.children[chosen_option.id]
            elif 'default' in self.current_node.children:
                next_node = self.current_node.children['default']

            self.current_node = next_node

        return {
            'status': 'completed',
            'decisions_made': len(self.decision_history),
            'final_context': self.workflow_context,
            'decision_path': [d['chosen_option'] for d in self.decision_history]
        }

# Create sample decision workflow
workflow = DecisionWorkflow("Software Development Planning")

# Node 1: Choose development approach
approach_problem = DecisionProblem(
    id="dev_approach",
    title="Development Approach",
    description="Choose software development methodology",
    decision_type=DecisionType.STRATEGIC,
    criteria=[
        DecisionCriterion("speed", "delivery_speed", "Time to delivery", 0.4, "maximize"),
        DecisionCriterion("quality", "code_quality", "Code quality assurance", 0.3, "maximize"),
        DecisionCriterion("flexibility", "adaptability", "Ability to change", 0.3, "maximize")
    ],
    options=[
        DecisionOption("agile", "Agile Development", "Iterative development",
                      {"delivery_speed": 80, "code_quality": 75, "adaptability": 90}),
        DecisionOption("waterfall", "Waterfall Method", "Sequential development",
                      {"delivery_speed": 60, "code_quality": 85, "adaptability": 40}),
        DecisionOption("devops", "DevOps Approach", "Continuous integration",
                      {"delivery_speed": 85, "code_quality": 80, "adaptability": 75})
    ]
)

node1 = DecisionNode("approach_decision", approach_problem)

# Callback to update context based on approach choice
def approach_callback(context, chosen_option):
    context['development_approach'] = chosen_option.id
    context['timeline_pressure'] = 'high' if chosen_option.id == 'agile' else 'medium'

node1.add_completion_callback(approach_callback)

# Node 2: Choose technology stack (depends on approach)
tech_problem = DecisionProblem(
    id="tech_stack",
    title="Technology Stack",
    description="Select primary technology stack",
    decision_type=DecisionType.TACTICAL,
    criteria=[
        DecisionCriterion("performance", "runtime_performance", "Execution performance", 0.3, "maximize"),
        DecisionCriterion("productivity", "dev_productivity", "Development speed", 0.4, "maximize"),
        DecisionCriterion("ecosystem", "library_support", "Available libraries", 0.3, "maximize")
    ],
    options=[
        DecisionOption("python", "Python Stack", "Python-based development",
                      {"runtime_performance": 70, "dev_productivity": 90, "library_support": 95}),
        DecisionOption("javascript", "JavaScript Stack", "Node.js development",
                      {"runtime_performance": 75, "dev_productivity": 85, "library_support": 90}),
        DecisionOption("java", "Java Stack", "Enterprise Java development",
                      {"runtime_performance": 85, "dev_productivity": 70, "library_support": 85})
    ]
)

node2 = DecisionNode("tech_decision", tech_problem)

# Node 3: Deployment strategy
deploy_problem = DecisionProblem(
    id="deployment",
    title="Deployment Strategy",
    description="Choose deployment and hosting approach",
    decision_type=DecisionType.OPERATIONAL,
    criteria=[
        DecisionCriterion("scalability", "scale_capability", "Scaling potential", 0.4, "maximize"),
        DecisionCriterion("cost", "operational_cost", "Monthly operating cost", 0.3, "minimize"),
        DecisionCriterion("maintenance", "ease_of_maintenance", "Maintenance burden", 0.3, "maximize")
    ],
    options=[
        DecisionOption("cloud", "Cloud Native", "Container-based cloud deployment",
                      {"scale_capability": 95, "operational_cost": 60, "ease_of_maintenance": 80}),
        DecisionOption("traditional", "Traditional Hosting", "Virtual machine deployment",
                      {"scale_capability": 70, "operational_cost": 40, "ease_of_maintenance": 60}),
        DecisionOption("hybrid", "Hybrid Approach", "Mixed cloud and on-premise",
                      {"scale_capability": 80, "operational_cost": 50, "ease_of_maintenance": 70})
    ]
)

node3 = DecisionNode("deploy_decision", deploy_problem)

# Connect workflow nodes
node1.add_child('agile', node2)
node1.add_child('devops', node2)
node1.add_child('waterfall', node2)
node1.add_child('default', node2)

node2.add_child('python', node3)
node2.add_child('javascript', node3)
node2.add_child('java', node3)
node2.add_child('default', node3)

# Set up workflow
workflow.set_root(node1)

# Execute workflow
workflow_agent = DeliberativeAgent("WorkflowAgent")
workflow_result = workflow.execute_workflow(workflow_agent)

print(f"   ✅ Workflow result: {workflow_result['status']}")
print(f"   📊 Decisions made: {workflow_result['decisions_made']}")
print(f"   🛤️  Decision path: {' → '.join(workflow_result['decision_path'])}")

print()


📝 Step 12: Decision Trees and Complex Workflows...
🧠 Created deliberative agent: WorkflowAgent
   🔄 Executing workflow: Software Development Planning
      📋 Current decision: Development Approach
🤔 WorkflowAgent deliberating on: Development Approach
   ✅ 3 valid options after constraints
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ 

In [25]:
# Step 13: Multi-agent decision coordination
print("📝 Step 13: Multi-Agent Decision Coordination...")

class DecisionCoordinator:
    def __init__(self, name: str):
        self.name = name
        self.agents = []
        self.coordination_strategy = "consensus"  # consensus, majority, weighted
        self.agent_weights = {}

    def add_agent(self, agent: DeliberativeAgent, weight: float = 1.0):
        self.agents.append(agent)
        self.agent_weights[agent.name] = weight
        print(f"   👥 Added agent {agent.name} with weight {weight}")

    def coordinate_decision(self, problem: DecisionProblem) -> Dict[str, Any]:
        """Coordinate decision across multiple agents"""
        print(f"   🤝 Coordinating decision: {problem.title}")

        agent_decisions = {}
        agent_explanations = {}

        # Get decision from each agent
        for agent in self.agents:
            result = agent.deliberate(problem, explain_decision=True)

            if 'error' not in result:
                agent_decisions[agent.name] = result['chosen_option']
                agent_explanations[agent.name] = result.get('explanation', {})
                print(f"      {agent.name}: {result['chosen_option'].name} (conf: {result['confidence']:.1%})")
            else:
                print(f"      {agent.name}: Failed to decide")

        if not agent_decisions:
            return {'error': 'No agents made successful decisions'}

        # Apply coordination strategy
        if self.coordination_strategy == "consensus":
            return self._consensus_decision(agent_decisions, agent_explanations)
        elif self.coordination_strategy == "majority":
            return self._majority_decision(agent_decisions, agent_explanations)
        elif self.coordination_strategy == "weighted":
            return self._weighted_decision(agent_decisions, agent_explanations)

    def _consensus_decision(self, decisions: Dict[str, DecisionOption],
                          explanations: Dict[str, Dict]) -> Dict[str, Any]:
        """Require unanimous agreement"""
        if len(set(d.id for d in decisions.values())) == 1:
            chosen = list(decisions.values())[0]
            return {
                'coordination_result': 'consensus_achieved',
                'chosen_option': chosen,
                'supporting_agents': list(decisions.keys()),
                'confidence': 1.0
            }
        else:
            return {
                'coordination_result': 'no_consensus',
                'fallback_option': max(decisions.values(), key=lambda x: x.overall_score),
                'disagreeing_agents': list(decisions.keys()),
                'confidence': 0.5
            }

    def _majority_decision(self, decisions: Dict[str, DecisionOption],
                         explanations: Dict[str, Dict]) -> Dict[str, Any]:
        """Use majority vote"""
        vote_counts = defaultdict(list)
        for agent_name, option in decisions.items():
            vote_counts[option.id].append(agent_name)

        winner = max(vote_counts.items(), key=lambda x: len(x[1]))
        winner_option = next(opt for opt in decisions.values() if opt.id == winner[0])

        return {
            'coordination_result': 'majority_decision',
            'chosen_option': winner_option,
            'supporting_agents': winner[1],
            'vote_count': len(winner[1]),
            'total_voters': len(decisions),
            'confidence': len(winner[1]) / len(decisions)
        }

    def _weighted_decision(self, decisions: Dict[str, DecisionOption],
                         explanations: Dict[str, Dict]) -> Dict[str, Any]:
        """Use weighted voting based on agent weights"""
        weighted_scores = defaultdict(float)
        total_weight = 0.0

        for agent_name, option in decisions.items():
            weight = self.agent_weights.get(agent_name, 1.0)
            weighted_scores[option.id] += option.overall_score * weight
            total_weight += weight

        # Normalize scores
        for option_id in weighted_scores:
            weighted_scores[option_id] /= total_weight

        winner_id = max(weighted_scores.items(), key=lambda x: x[1])
        winner_option = next(opt for opt in decisions.values() if opt.id == winner_id[0])

        return {
            'coordination_result': 'weighted_decision',
            'chosen_option': winner_option,
            'weighted_score': winner_id[1],
            'contributing_agents': list(decisions.keys()),
            'confidence': winner_id[1]
        }

# Test multi-agent coordination
coordinator = DecisionCoordinator("TechDecisionCoordinator")

# Create diverse agents
conservative_agent = DeliberativeAgent("ConservativeExpert")
conservative_agent.risk_tolerance = 0.2
conservative_agent.set_preferences({"cost": 1.5, "risk": 0.5})

innovative_agent = DeliberativeAgent("InnovativeExpert")
innovative_agent.risk_tolerance = 0.8
innovative_agent.set_preferences({"performance": 1.3, "scalability": 1.2})

balanced_agent = DeliberativeAgent("BalancedExpert")
balanced_agent.risk_tolerance = 0.5

# Add agents with different weights
coordinator.add_agent(conservative_agent, weight=1.0)
coordinator.add_agent(innovative_agent, weight=1.2)  # Slightly higher weight
coordinator.add_agent(balanced_agent, weight=1.0)

# Test different coordination strategies
test_problem = create_investment_decision_problem()

for strategy in ["consensus", "majority", "weighted"]:
    coordinator.coordination_strategy = strategy
    print(f"\n   🎯 Testing {strategy} coordination:")

    result = coordinator.coordinate_decision(test_problem)

    if 'error' not in result:
        print(f"      Result: {result['coordination_result']}")
        print(f"      Chosen: {result['chosen_option'].name}")
        print(f"      Confidence: {result['confidence']:.1%}")
    else:
        print(f"      Failed: {result['error']}")

print()

📝 Step 13: Multi-Agent Decision Coordination...
🧠 Created deliberative agent: ConservativeExpert
   📝 Updated preferences: 2 criteria
🧠 Created deliberative agent: InnovativeExpert
   📝 Updated preferences: 2 criteria
🧠 Created deliberative agent: BalancedExpert
   👥 Added agent ConservativeExpert with weight 1.0
   👥 Added agent InnovativeExpert with weight 1.2
   👥 Added agent BalancedExpert with weight 1.0

   🎯 Testing consensus coordination:
   🤝 Coordinating decision: Investment Portfolio Allocation
🤔 ConservativeExpert deliberating on: Investment Portfolio Allocation
   ✅ 4 valid options after constraints
     🎯 Adjusted risk_level weight by 0.50x
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria.

In [26]:
# Step 14: Decision quality metrics and validation
print("📝 Step 14: Decision Quality Metrics and Validation...")

class DecisionQualityAnalyzer:
    def __init__(self):
        self.quality_metrics = {}
        self.validation_tests = []

    def add_quality_metric(self, name: str, metric_func: Callable):
        """Add a quality assessment metric"""
        self.quality_metrics[name] = metric_func

    def add_validation_test(self, name: str, test_func: Callable):
        """Add a validation test"""
        self.validation_tests.append((name, test_func))

    def assess_decision_quality(self, problem: DecisionProblem,
                              chosen_option: DecisionOption,
                              decision_process: Dict[str, Any]) -> Dict[str, Any]:
        """Comprehensive decision quality assessment"""

        quality_scores = {}

        # Run quality metrics
        for metric_name, metric_func in self.quality_metrics.items():
            try:
                score = metric_func(problem, chosen_option, decision_process)
                quality_scores[metric_name] = score
            except Exception as e:
                quality_scores[metric_name] = f"Error: {e}"

        # Run validation tests
        validation_results = {}
        for test_name, test_func in self.validation_tests:
            try:
                result = test_func(problem, chosen_option, decision_process)
                validation_results[test_name] = result
            except Exception as e:
                validation_results[test_name] = f"Error: {e}"

        # Calculate overall quality score
        numeric_scores = [s for s in quality_scores.values() if isinstance(s, (int, float))]
        overall_quality = sum(numeric_scores) / len(numeric_scores) if numeric_scores else 0.0

        return {
            'overall_quality': overall_quality,
            'quality_metrics': quality_scores,
            'validation_results': validation_results,
            'quality_grade': self._grade_quality(overall_quality)
        }

    def _grade_quality(self, score: float) -> str:
        """Convert quality score to letter grade"""
        if score >= 0.9: return "A"
        elif score >= 0.8: return "B"
        elif score >= 0.7: return "C"
        elif score >= 0.6: return "D"
        else: return "F"

# Create quality analyzer with metrics
quality_analyzer = DecisionQualityAnalyzer()

# Define quality metrics
quality_analyzer.add_quality_metric(
    "option_coverage",
    lambda prob, opt, proc: len(prob.options) / max(5, len(prob.options))  # Prefer more options considered
)

quality_analyzer.add_quality_metric(
    "criteria_balance",
    lambda prob, opt, proc: 1.0 - max(c.weight for c in prob.criteria)  # Prefer balanced criteria
)

quality_analyzer.add_quality_metric(
    "decision_confidence",
    lambda prob, opt, proc: proc.get('confidence', 0.5)  # Higher confidence is better
)

quality_analyzer.add_quality_metric(
    "score_separation",
    lambda prob, opt, proc: opt.overall_score - sorted([o.overall_score for o in prob.options])[-2]  # Clear winner
)

# Define validation tests
quality_analyzer.add_validation_test(
    "constraint_compliance",
    lambda prob, opt, proc: all(c.get('satisfied', True) for c in proc.get('constraints_checked', []))
)

quality_analyzer.add_validation_test(
    "strategy_appropriateness",
    lambda prob, opt, proc: proc.get('reasoning_strategy') != 'intuitive' or prob.decision_type == DecisionType.EMERGENCY
)

# Test decision quality assessment
test_agent = DeliberativeAgent("QualityTestAgent")
test_problem = create_sample_decision_problem()

decision_result = test_agent.deliberate(test_problem, explain_decision=True)

if 'error' not in decision_result:
    quality_assessment = quality_analyzer.assess_decision_quality(
        test_problem,
        decision_result['chosen_option'],
        decision_result
    )

    print("   📊 Decision Quality Assessment:")
    print(f"      Overall Quality: {quality_assessment['overall_quality']:.2f} (Grade: {quality_assessment['quality_grade']})")
    print(f"      Quality Metrics:")
    for metric, score in quality_assessment['quality_metrics'].items():
        if isinstance(score, (int, float)):
            print(f"        {metric}: {score:.2f}")
        else:
            print(f"        {metric}: {score}")

    print(f"      Validation Results:")
    for test, result in quality_assessment['validation_results'].items():
        status = "✅" if result else "❌"
        print(f"        {test}: {status}")

print()


📝 Step 14: Decision Quality Metrics and Validation...
🧠 Created deliberative agent: QualityTestAgent
🤔 QualityTestAgent deliberating on: Choose a Laptop for Work
   ✅ 4 valid options after constraints
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation compl

In [27]:
# Step 15: Integration with external systems and APIs
print("📝 Step 15: External System Integration...")

class ExternalDataProvider:
    """Simulates integration with external data sources"""

    def __init__(self, name: str):
        self.name = name
        self.data_cache = {}
        self.api_calls = 0

    def get_market_data(self, asset: str) -> Dict[str, Any]:
        """Simulate getting market data"""
        self.api_calls += 1

        # Simulate API response
        market_data = {
            'price': random.uniform(100, 1000),
            'volatility': random.uniform(0.1, 0.5),
            'trend': random.choice(['bullish', 'bearish', 'neutral']),
            'volume': random.randint(1000, 10000),
            'timestamp': time.time()
        }

        self.data_cache[asset] = market_data
        return market_data

    def get_weather_data(self, location: str) -> Dict[str, Any]:
        """Simulate getting weather data"""
        self.api_calls += 1

        weather_data = {
            'temperature': random.uniform(50, 90),
            'humidity': random.uniform(30, 90),
            'precipitation_chance': random.uniform(0, 100),
            'wind_speed': random.uniform(0, 30),
            'visibility': random.uniform(1, 10)
        }

        return weather_data

    def get_user_preferences(self, user_id: str) -> Dict[str, Any]:
        """Simulate getting user preference data"""
        self.api_calls += 1

        preferences = {
            'risk_tolerance': random.uniform(0.2, 0.8),
            'preferred_categories': random.sample(['tech', 'finance', 'health', 'entertainment'], 2),
            'budget_range': (random.randint(500, 1000), random.randint(1500, 3000)),
            'priority_factors': random.sample(['cost', 'quality', 'speed', 'reliability'], 3)
        }

        return preferences

class DataIntegratedAgent(DeliberativeAgent):
    """Agent that integrates external data into decision making"""

    def __init__(self, name: str, data_provider: ExternalDataProvider):
        super().__init__(name)
        self.data_provider = data_provider
        self.external_data_cache = {}

    def deliberate_with_external_data(self, problem: DecisionProblem,
                                    data_requirements: List[str] = None) -> Dict[str, Any]:
        """Enhanced deliberation with external data integration"""

        print(f"   🌐 {self.name} gathering external data...")

        # Gather required external data
        external_data = {}
        if data_requirements:
            for requirement in data_requirements:
                if requirement == 'market':
                    external_data['market'] = self.data_provider.get_market_data('tech_index')
                elif requirement == 'weather':
                    external_data['weather'] = self.data_provider.get_weather_data('headquarters')
                elif requirement == 'user_prefs':
                    external_data['user_prefs'] = self.data_provider.get_user_preferences('current_user')

        # Integrate external data into criteria weights
        self._integrate_external_data(problem, external_data)

        # Perform standard deliberation
        result = self.deliberate(problem, explain_decision=False)

        # Add external data context to result
        if 'error' not in result:
            result['external_data_used'] = external_data
            result['data_integration_notes'] = self._generate_data_notes(external_data)

        return result

    def _integrate_external_data(self, problem: DecisionProblem, external_data: Dict[str, Any]):
        """Modify problem criteria based on external data"""

        if 'market' in external_data:
            market = external_data['market']
            # Adjust risk-related criteria based on market volatility
            for criterion in problem.criteria:
                if 'risk' in criterion.name.lower():
                    if market['volatility'] > 0.3:  # High volatility
                        criterion.weight *= 1.2  # Increase risk consideration
                    elif market['volatility'] < 0.15:  # Low volatility
                        criterion.weight *= 0.8  # Decrease risk consideration

        if 'weather' in external_data:
            weather = external_data['weather']
            # Adjust timeline criteria based on weather (example: outdoor events)
            for criterion in problem.criteria:
                if 'time' in criterion.name.lower() or 'speed' in criterion.name.lower():
                    if weather['precipitation_chance'] > 70:
                        criterion.weight *= 1.1  # Timeline more important in bad weather

        if 'user_prefs' in external_data:
            prefs = external_data['user_prefs']
            # Adjust criteria weights based on user preferences
            self.risk_tolerance = prefs['risk_tolerance']

            # Update preferences for specific factors
            for factor in prefs['priority_factors']:
                for criterion in problem.criteria:
                    if factor in criterion.name.lower():
                        criterion.weight *= 1.3

        # Renormalize weights
        total_weight = sum(c.weight for c in problem.criteria)
        if total_weight > 0:
            for criterion in problem.criteria:
                criterion.weight /= total_weight

    def _generate_data_notes(self, external_data: Dict[str, Any]) -> List[str]:
        """Generate human-readable notes about data integration"""
        notes = []

        if 'market' in external_data:
            market = external_data['market']
            notes.append(f"Market volatility: {market['volatility']:.1%} - {'High' if market['volatility'] > 0.3 else 'Low'} risk environment")

        if 'weather' in external_data:
            weather = external_data['weather']
            notes.append(f"Weather conditions: {weather['precipitation_chance']:.0f}% precipitation chance")

        if 'user_prefs' in external_data:
            prefs = external_data['user_prefs']
            notes.append(f"User risk tolerance: {prefs['risk_tolerance']:.1%}")
            notes.append(f"Priority factors: {', '.join(prefs['priority_factors'])}")

        return notes

# Test external data integration
data_provider = ExternalDataProvider("MockAPI")
integrated_agent = DataIntegratedAgent("DataAwareAgent", data_provider)

test_problem = create_investment_decision_problem()
data_requirements = ['market', 'user_prefs']

print("   🔗 Testing external data integration:")
integrated_result = integrated_agent.deliberate_with_external_data(test_problem, data_requirements)

if 'error' not in integrated_result:
    print(f"      Chosen: {integrated_result['chosen_option'].name}")
    print(f"      Confidence: {integrated_result['confidence']:.1%}")
    print(f"      API calls made: {data_provider.api_calls}")

    if integrated_result['data_integration_notes']:
        print(f"      Data integration notes:")
        for note in integrated_result['data_integration_notes']:
            print(f"        • {note}")

print()


📝 Step 15: External System Integration...
🧠 Created deliberative agent: DataAwareAgent
   🔗 Testing external data integration:
   🌐 DataAwareAgent gathering external data...
🤔 DataAwareAgent deliberating on: Investment Portfolio Allocation
   ✅ 4 valid options after constraints
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation complete
   📊 Evaluating 1 options against 3 criteria...
   ✅ Evaluation 

In [28]:
# Step 16: Real-time decision monitoring and adaptation
print("📝 Step 16: Real-time Decision Monitoring...")

class DecisionMonitor:
    """Monitor ongoing decisions and trigger adaptations"""

    def __init__(self):
        self.active_decisions = {}
        self.monitoring_rules = []
        self.adaptation_triggers = []

    def register_decision(self, decision_id: str, problem: DecisionProblem,
                         chosen_option: DecisionOption, monitor_duration: int = 300):
        """Register a decision for monitoring"""
        self.active_decisions[decision_id] = {
            'problem': problem,
            'chosen_option': chosen_option,
            'start_time': time.time(),
            'monitor_until': time.time() + monitor_duration,
            'status': 'active',
            'metrics': defaultdict(list)
        }
        print(f"   📡 Monitoring decision {decision_id} for {monitor_duration}s")

    def add_monitoring_rule(self, rule_name: str, condition_func: Callable,
                           action_func: Callable):
        """Add a monitoring rule"""
        self.monitoring_rules.append({
            'name': rule_name,
            'condition': condition_func,
            'action': action_func
        })

    def update_decision_metrics(self, decision_id: str, metrics: Dict[str, float]):
        """Update metrics for a monitored decision"""
        if decision_id in self.active_decisions:
            decision = self.active_decisions[decision_id]
            for metric_name, value in metrics.items():
                decision['metrics'][metric_name].append((time.time(), value))

    def check_monitoring_rules(self) -> List[Dict[str, Any]]:
        """Check all monitoring rules against active decisions"""
        triggered_actions = []

        current_time = time.time()

        for decision_id, decision in self.active_decisions.items():
            # Skip expired decisions
            if current_time > decision['monitor_until']:
                decision['status'] = 'expired'
                continue

            # Check each monitoring rule
            for rule in self.monitoring_rules:
                try:
                    if rule['condition'](decision_id, decision):
                        action_result = rule['action'](decision_id, decision)
                        triggered_actions.append({
                            'decision_id': decision_id,
                            'rule_name': rule['name'],
                            'action_result': action_result,
                            'timestamp': current_time
                        })
                except Exception as e:
                    print(f"     ⚠️ Error in monitoring rule {rule['name']}: {e}")

        return triggered_actions

    def get_decision_status(self, decision_id: str) -> Dict[str, Any]:
        """Get current status of a monitored decision"""
        if decision_id not in self.active_decisions:
            return {'error': 'Decision not found'}

        decision = self.active_decisions[decision_id]
        current_time = time.time()

        # Calculate performance metrics
        performance_summary = {}
        for metric_name, values in decision['metrics'].items():
            if values:
                recent_values = [v[1] for v in values[-5:]]  # Last 5 values
                performance_summary[metric_name] = {
                    'current': values[-1][1],
                    'average': sum(recent_values) / len(recent_values),
                    'trend': 'improving' if len(values) > 1 and values[-1][1] > values[-2][1] else 'declining'
                }

        return {
            'decision_id': decision_id,
            'status': decision['status'],
            'chosen_option': decision['chosen_option'].name,
            'elapsed_time': current_time - decision['start_time'],
            'remaining_monitor_time': max(0, decision['monitor_until'] - current_time),
            'performance_metrics': performance_summary
        }

# Set up decision monitoring
monitor = DecisionMonitor()

# Add monitoring rules
monitor.add_monitoring_rule(
    "performance_degradation",
    lambda did, decision: (
        'performance' in decision['metrics'] and
        len(decision['metrics']['performance']) > 3 and
        decision['metrics']['performance'][-1][1] < 0.5
    ),
    lambda did, decision: f"Alert: Performance below threshold for {did}"
)

monitor.add_monitoring_rule(
    "cost_overrun",
    lambda did, decision: (
        'cost' in decision['metrics'] and
        any(cost[1] > 1000 for cost in decision['metrics']['cost'][-3:])
    ),
    lambda did, decision: f"Alert: Cost overrun detected for {did}"
)

# Test decision monitoring
test_decision_id = "monitor_test_001"
test_problem = create_sample_decision_problem()
test_chosen = test_problem.options[0]

monitor.register_decision(test_decision_id, test_problem, test_chosen, monitor_duration=60)

# Simulate some metrics updates
print("   📊 Simulating decision monitoring:")
for i in range(5):
    time.sleep(0.5)  # Small delay for simulation

    # Simulate changing metrics
    performance = 0.8 - (i * 0.1)  # Declining performance
    cost = 500 + (i * 100)  # Rising cost

    monitor.update_decision_metrics(test_decision_id, {
        'performance': performance,
        'cost': cost,
        'user_satisfaction': random.uniform(0.6, 0.9)
    })

    # Check monitoring rules
    triggered = monitor.check_monitoring_rules()
    if triggered:
        for action in triggered:
            print(f"      🚨 {action['rule_name']}: {action['action_result']}")

    # Show current status
    status = monitor.get_decision_status(test_decision_id)
    print(f"      Status at {i+1}: Performance {status['performance_metrics'].get('performance', {}).get('current', 'N/A')}")

print()


📝 Step 16: Real-time Decision Monitoring...
   📡 Monitoring decision monitor_test_001 for 60s
   📊 Simulating decision monitoring:
      Status at 1: Performance 0.8
      Status at 2: Performance 0.7000000000000001
      Status at 3: Performance 0.6000000000000001
      Status at 4: Performance 0.5
      🚨 performance_degradation: Alert: Performance below threshold for monitor_test_001
      Status at 5: Performance 0.4



In [29]:
# Step 17: Final comprehensive example
print("📝 Step 17: Comprehensive Decision-Making Example...")

class ComprehensiveDecisionSystem:
    """Complete decision-making system integrating all components"""

    def __init__(self, name: str):
        self.name = name
        self.agents = {}
        self.coordinator = None
        self.monitor = DecisionMonitor()
        self.explainer = DecisionExplainer()
        self.quality_analyzer = DecisionQualityAnalyzer()
        self.data_provider = ExternalDataProvider("SystemAPI")

        print(f"🏭 Initialized comprehensive decision system: {name}")

    def add_decision_agent(self, agent: DeliberativeAgent, role: str):
        """Add a specialized decision agent"""
        self.agents[role] = agent
        print(f"   👤 Added {role} agent: {agent.name}")

    def set_coordinator(self, coordinator: DecisionCoordinator):
        """Set the coordination system"""
        self.coordinator = coordinator

    def process_complex_decision(self, problem: DecisionProblem,
                                context: Dict[str, Any] = None) -> Dict[str, Any]:
        """Process a complex decision with full system capabilities"""

        print(f"\n🏭 Processing complex decision: {problem.title}")
        print("=" * 50)

        context = context or {}
        decision_id = f"decision_{int(time.time())}"

        # Step 1: External data gathering
        print("📡 1. Gathering external data...")
        external_data = {}
        if context.get('require_market_data'):
            external_data['market'] = self.data_provider.get_market_data('index')
        if context.get('require_user_data'):
            external_data['user'] = self.data_provider.get_user_preferences('current')

        # Step 2: Agent coordination
        print("🤝 2. Coordinating agent decisions...")
        if self.coordinator and len(self.agents) > 1:
            coordination_result = self.coordinator.coordinate_decision(problem)
            chosen_option = coordination_result.get('chosen_option')
            decision_confidence = coordination_result.get('confidence', 0.5)
            coordination_method = coordination_result.get('coordination_result')
        else:
            # Single agent decision
            primary_agent = list(self.agents.values())[0] if self.agents else DeliberativeAgent("DefaultAgent")
            agent_result = primary_agent.deliberate(problem)
            chosen_option = agent_result.get('chosen_option')
            decision_confidence = agent_result.get('confidence', 0.5)
            coordination_method = "single_agent"

        if not chosen_option:
            return {'error': 'No decision could be made'}

        # Step 3: Decision quality assessment
        print("📊 3. Assessing decision quality...")
        quality_result = self.quality_analyzer.assess_decision_quality(
            problem, chosen_option, {
                'confidence': decision_confidence,
                'reasoning_strategy': 'coordinated',
                'external_data': external_data
            }
        )

        # Step 4: Generate explanation
        print("📝 4. Generating decision explanation...")
        explanation = self.explainer.explain_decision_process(
            problem, chosen_option, ReasoningStrategy.ANALYTICAL, detailed=True
        )

        # Step 5: Set up monitoring
        print("📡 5. Setting up decision monitoring...")
        if context.get('monitor_decision', True):
            self.monitor.register_decision(
                decision_id, problem, chosen_option,
                monitor_duration=context.get('monitor_duration', 300)
            )

        # Compile comprehensive result
        result = {
            'decision_id': decision_id,
            'chosen_option': chosen_option,
            'confidence': decision_confidence,
            'coordination_method': coordination_method,
            'quality_assessment': quality_result,
            'explanation': explanation,
            'external_data_used': external_data,
            'monitoring_active': context.get('monitor_decision', True),
            'system_performance': {
                'agents_involved': len(self.agents),
                'api_calls_made': self.data_provider.api_calls,
                'decision_timestamp': time.time()
            }
        }

        print(f"✅ Decision complete: {chosen_option.name}")
        print(f"   Quality Grade: {quality_result['quality_grade']}")
        print(f"   Confidence: {decision_confidence:.1%}")
        print(f"   Coordination: {coordination_method}")

        return result

# Create comprehensive system
decision_system = ComprehensiveDecisionSystem("Enterprise Decision Platform")

# Add specialized agents
financial_agent = DeliberativeAgent("FinancialAnalyst")
financial_agent.set_preferences({"cost": 1.3, "roi": 1.5, "risk": 0.7})

technical_agent = DeliberativeAgent("TechnicalExpert")
technical_agent.set_preferences({"performance": 1.4, "scalability": 1.2, "maintenance": 1.1})

user_agent = DeliberativeAgent("UserAdvocate")
user_agent.set_preferences({"usability": 1.5, "satisfaction": 1.3})

decision_system.add_decision_agent(financial_agent, "financial")
decision_system.add_decision_agent(technical_agent, "technical")
decision_system.add_decision_agent(user_agent, "user_experience")

# Set up coordination
system_coordinator = DecisionCoordinator("SystemCoordinator")
system_coordinator.add_agent(financial_agent, weight=1.0)
system_coordinator.add_agent(technical_agent, weight=1.2)
system_coordinator.add_agent(user_agent, weight=1.1)
system_coordinator.coordination_strategy = "weighted"

decision_system.set_coordinator(system_coordinator)

# Process complex decision
complex_problem = DecisionProblem(
    id="platform_migration",
    title="Enterprise Platform Migration",
    description="Choose new enterprise platform architecture",
    decision_type=DecisionType.STRATEGIC,
    criteria=[
        DecisionCriterion("cost", "total_cost", "Total migration cost", 0.25, "minimize"),
        DecisionCriterion("performance", "system_performance", "System performance", 0.25, "maximize"),
        DecisionCriterion("risk", "migration_risk", "Migration risk level", 0.2, "minimize"),
        DecisionCriterion("scalability", "future_scalability", "Future scaling capability", 0.15, "maximize"),
        DecisionCriterion("timeline", "implementation_time", "Time to implement", 0.15, "minimize")
    ],
    options=[
        DecisionOption("cloud_native", "Cloud Native Solution", "Full cloud transformation",
                      {"total_cost": 2000000, "system_performance": 90, "migration_risk": 7,
                       "future_scalability": 95, "implementation_time": 18}),
        DecisionOption("hybrid", "Hybrid Approach", "Mixed cloud and on-premise",
                      {"total_cost": 1500000, "system_performance": 80, "migration_risk": 5,
                       "future_scalability": 80, "implementation_time": 12}),
        DecisionOption("modernize", "Modernize Current", "Upgrade existing systems",
                      {"total_cost": 800000, "system_performance": 70, "migration_risk": 3,
                       "future_scalability": 60, "implementation_time": 8}),
        DecisionOption("gradual", "Gradual Migration", "Phased migration approach",
                      {"total_cost": 1200000, "system_performance": 75, "migration_risk": 4,
                       "future_scalability": 75, "implementation_time": 24})
    ]
)

# Process the complex decision
complex_context = {
    'require_market_data': True,
    'require_user_data': True,
    'monitor_decision': True,
    'monitor_duration': 600
}

final_result = decision_system.process_complex_decision(complex_problem, complex_context)

print(f"\n📋 COMPREHENSIVE DECISION SUMMARY")
print("=" * 40)
print(f"Decision ID: {final_result['decision_id']}")
print(f"Final Choice: {final_result['chosen_option'].name}")
print(f"Overall Confidence: {final_result['confidence']:.1%}")
print(f"Quality Grade: {final_result['quality_assessment']['quality_grade']}")
print(f"Agents Involved: {final_result['system_performance']['agents_involved']}")
print(f"Coordination Method: {final_result['coordination_method']}")

print()


📝 Step 17: Comprehensive Decision-Making Example...
🏭 Initialized comprehensive decision system: Enterprise Decision Platform
🧠 Created deliberative agent: FinancialAnalyst
   📝 Updated preferences: 3 criteria
🧠 Created deliberative agent: TechnicalExpert
   📝 Updated preferences: 3 criteria
🧠 Created deliberative agent: UserAdvocate
   📝 Updated preferences: 2 criteria
   👤 Added financial agent: FinancialAnalyst
   👤 Added technical agent: TechnicalExpert
   👤 Added user_experience agent: UserAdvocate
   👥 Added agent FinancialAnalyst with weight 1.0
   👥 Added agent TechnicalExpert with weight 1.2
   👥 Added agent UserAdvocate with weight 1.1

🏭 Processing complex decision: Enterprise Platform Migration
📡 1. Gathering external data...
🤝 2. Coordinating agent decisions...
   🤝 Coordinating decision: Enterprise Platform Migration
🤔 FinancialAnalyst deliberating on: Enterprise Platform Migration
   ✅ 4 valid options after constraints
     🎯 Adjusted total_cost weight by 1.30x
     🎯 Ad

In [30]:
# TUTORIAL COMPLETION AND SUMMARY
# =============================================================================

print("✅ Tutorial 11 Complete!")

print()

print("📚 WHAT WE LEARNED:")

print("=" * 40)

print("1. 🧠 Multi-criteria decision analysis")

print("   - Structured problem representation")

print("   - Weighted evaluation frameworks")

print("   - Option scoring and ranking")

print()

print("2. 🎯 Multiple reasoning strategies")

print("   - Analytical: thorough systematic analysis")

print("   - Intuitive: fast heuristic-based decisions")

print("   - Satisficing: first acceptable solution")

print("   - Optimizing: best possible solution")

print()

print("3. 🤖 Learning and adaptive agents")

print("   - Experience-based strategy selection")

print("   - Pattern recognition in decision contexts")

print("   - Performance tracking and improvement")

print()

print("4. 🔍 Advanced reasoning capabilities")

print("   - Uncertainty handling and robustness analysis")

print("   - Constraint satisfaction and filtering")

print("   - Risk assessment and tolerance modeling")

print()

print("5. 🤝 Multi-agent coordination")

print("   - Consensus and majority decision making")

print("   - Weighted voting and expert systems")

print("   - Conflict resolution and negotiation")

print()

print("6. 📊 Decision transparency and explanation")

print("   - Comprehensive decision documentation")

print("   - Quality assessment and validation")

print("   - Human-readable explanations")

print()

print("7. 🌐 External system integration")

print("   - Real-time data incorporation")

print("   - API-driven decision enhancement")

print("   - Context-aware adaptations")

print()

print("8. 📡 Monitoring and continuous improvement")

print("   - Real-time decision monitoring")

print("   - Performance tracking and alerts")

print("   - Adaptive system responses")

print()


✅ Tutorial 11 Complete!

📚 WHAT WE LEARNED:
1. 🧠 Multi-criteria decision analysis
   - Structured problem representation
   - Weighted evaluation frameworks
   - Option scoring and ranking

2. 🎯 Multiple reasoning strategies
   - Analytical: thorough systematic analysis
   - Intuitive: fast heuristic-based decisions
   - Satisficing: first acceptable solution
   - Optimizing: best possible solution

3. 🤖 Learning and adaptive agents
   - Experience-based strategy selection
   - Pattern recognition in decision contexts
   - Performance tracking and improvement

4. 🔍 Advanced reasoning capabilities
   - Uncertainty handling and robustness analysis
   - Constraint satisfaction and filtering
   - Risk assessment and tolerance modeling

5. 🤝 Multi-agent coordination
   - Consensus and majority decision making
   - Weighted voting and expert systems
   - Conflict resolution and negotiation

6. 📊 Decision transparency and explanation
   - Comprehensive decision documentation
   - Quality asse

In [31]:
print("⚠️  COMMON ERRORS AND SOLUTIONS:")

print("=" * 40)

print("1. 🐛 Criteria weight imbalance")

print("   Problem: One criterion dominates decision making")

print("   Solution: Normalize weights and validate balance")

print("   Solution: Use sensitivity analysis to test robustness")

print()

print("2. 🐛 Strategy selection mismatches")

print("   Problem: Wrong reasoning strategy for decision type")

print("   Solution: Implement meta-reasoning for strategy choice")

print("   Solution: Learn from strategy performance over time")

print()

print("3. 🐛 Constraint conflicts and impossibility")

print("   Problem: No options satisfy all hard constraints")

print("   Solution: Implement constraint relaxation mechanisms")

print("   Solution: Use soft constraints with penalty systems")

print()

print("4. 🐛 Coordination deadlocks")

print("   Problem: Agents cannot reach consensus")

print("   Solution: Implement fallback decision mechanisms")

print("   Solution: Use weighted voting or expert arbitration")

print()

print("5. 🐛 External data staleness")

print("   Problem: Using outdated external information")

print("   Solution: Implement data freshness validation")

print("   Solution: Cache management and refresh policies")

print()

print("6. 🐛 Explanation complexity")

print("   Problem: Decision explanations too complex for users")

print("   Solution: Generate multi-level explanations")

print("   Solution: Focus on key factors and trade-offs")

print()

print("7. 🐛 Monitoring overhead")

print("   Problem: Too much monitoring impacts performance")

print("   Solution: Selective monitoring based on decision importance")

print("   Solution: Efficient metric collection and processing")

print()

print("🎉 Ready for Tutorial 12: Learning from Experience!")

print("   Next we'll explore how agents improve through experience...")

print("\n🌟 You've now mastered:")

print("   • Individual agent intelligence and communication")

print("   • Multi-agent networks and collaborative processing")

print("   • Production monitoring, configuration, and management")

print("   • Structured agent circuits for complex workflows")

print("   • Sophisticated memory systems for context and learning")

print("   • Advanced deliberative reasoning and decision making")

print("\n🚀 Ready to build truly intelligent decision-making systems!")

⚠️  COMMON ERRORS AND SOLUTIONS:
1. 🐛 Criteria weight imbalance
   Problem: One criterion dominates decision making
   Solution: Normalize weights and validate balance
   Solution: Use sensitivity analysis to test robustness

2. 🐛 Strategy selection mismatches
   Problem: Wrong reasoning strategy for decision type
   Solution: Implement meta-reasoning for strategy choice
   Solution: Learn from strategy performance over time

3. 🐛 Constraint conflicts and impossibility
   Problem: No options satisfy all hard constraints
   Solution: Implement constraint relaxation mechanisms
   Solution: Use soft constraints with penalty systems

4. 🐛 Coordination deadlocks
   Problem: Agents cannot reach consensus
   Solution: Implement fallback decision mechanisms
   Solution: Use weighted voting or expert arbitration

5. 🐛 External data staleness
   Problem: Using outdated external information
   Solution: Implement data freshness validation
   Solution: Cache management and refresh policies

6. 🐛 E

---

🔒 **INTELLECTUAL PROPERTY & LICENSE NOTICE**

This tutorial and its contents — including code, architecture, narrative examples, and educational structure — are the intellectual property of **Shalini Ananda, PhD** and part of the **Neuron Framework** under a **Modified MIT License with Attribution**.

- Commercial use, redistribution, or derivative works **must** include clear and visible attribution to the original author.
- Use in products, consulting engagements, or educational materials **must reference this repository and author name.**
- Removal of author credit or misrepresentation of origin constitutes **a violation of the license and may trigger legal action.**
- You may **not white-label, obfuscate, or rebrand** this work without explicit, written permission.

Use of this tutorial in Colab or any other platform implies agreement with these terms.

📘 **License**: [LICENSE.md](../LICENSE.md)  
📌 **Notice**: [NOTICE.md](../NOTICE.md)  
🧠 **Author**: [Shalini Ananda, PhD](https://github.com/ShaliniAnandaPhD)